In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import warnings
import random
warnings.filterwarnings("ignore")

!pip install bert-score
!pip install transformers
!pip install sentencepiece
from transformers import MarianMTModel, MarianTokenizer

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
/kaggle/input/data-train-bt-kompre/Data_train (80).csv


## DATA ##

In [2]:
data=pd.read_csv('/kaggle/input/data-train-bt-kompre/Data_train (80).csv')
data

,statement,status
0,i ve thought of ending my life so many time bu...,Depression
1,there is nothing to look forward to just the s...,Depression
2,just realized im talking to no one,Normal
3,husband stayed home from work today again hes ...,Bipolar
4,it feels like i am being punished for having i...,Depression
...,...,...
14916,tw suicide if i never kill myself all of this ...,Bipolar
14917,i find myself extremely less productive these ...,Depression
14918,what s the best way to eat reheated pizza i ke...,Normal
14919,its am seroquel reduction is going really well...,Bipolar


In [3]:
data.isnull().sum()

statement    0
status       0
dtype: int64

In [4]:
data.duplicated().sum()

0

In [5]:

data['status'].value_counts()

status
Normal                  4904
Depression              4469
Anxiety                 2164
Bipolar                 1457
Stress                  1401
Personality disorder     526
Name: count, dtype: int64

In [6]:
contoh_statements = data.groupby('status')['statement'].apply(lambda x: x.sample(n=1))

# Print the results with the index
for status, statement in contoh_statements.items():
    index = contoh_statements.index[contoh_statements == statement][0]  # Ambil index baris yang diambil
    print(f"Status: {status}")
    print(f"Statement: {statement}\n")

Status: ('Anxiety', 2869)
Statement: betoning the symptoms you read about some time ago i was worried i had a heart diseasd then i read about the symptoms and after i read about it i started having chest pains for the last months i have had a swollen lyntog node behind the left ear and after i read about the symptoms i started having low grade fever red cheeks night sweats sometimes and fatigue does anybody else become there symptoms

Status: ('Bipolar', 10827)
Statement: do you ever walk around your home for hours doing nothing i do this often and then its like i snap out of it and realize hours have gone by

Status: ('Depression', 13739)
Statement: if you would see me anywhere you would probably think i am happy because of my kindness and smiling and that i am handsome because i workout or whatever i do not really care but none of it is true none of anything is true with me i am dying inside at almost all times and if i am around people and happy those are the times when i am most sa

## Depression ##

In [7]:
import torch
# Pilih GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model names for translation
first_model_name = 'Helsinki-NLP/opus-mt-en-id'
second_model_name = 'Helsinki-NLP/opus-mt-id-en'

# Load the tokenizers
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained models and move them to GPU
first_model = MarianMTModel.from_pretrained(first_model_name).to(device)
second_model = MarianMTModel.from_pretrained(second_model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/796k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/291M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/291M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
# Filter the data for 'Depression' status
data_Depression = data[data['status'] == 'Depression']
original_texts = data_Depression['statement'].tolist()
data_Depression

,statement,status
0,i ve thought of ending my life so many time bu...,Depression
1,there is nothing to look forward to just the s...,Depression
4,it feels like i am being punished for having i...,Depression
5,because i always want to fucking kms i sexuall...,Depression
7,i think the pandemic hit me hard i moved out o...,Depression
...,...,...
14901,i stopped my dad from killing himself yesterda...,Depression
14904,are they trying i have read many scary stories...,Depression
14911,i had so many suicidal thought last night whil...,Depression
14917,i find myself extremely less productive these ...,Depression


In [9]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

# Kumpulan frasa depresi sebagai acuan makna
reference_texts = [
    "I want to die", "I feel hopeless", "Life is meaningless",
    "I'm tired of everything", "I feel empty and alone"
]
ref_embedding = model.encode(reference_texts, convert_to_tensor=True).mean(dim=0)

# Hitung embedding seluruh statement
texts = data_Depression['statement'].tolist()
text_embeddings = model.encode(texts, convert_to_tensor=True)

# Hitung similarity
similarities = util.pytorch_cos_sim(text_embeddings, ref_embedding).squeeze().tolist()
data_Depression['similarity_score'] = similarities

# Ambil 100 teks yang paling mirip dengan ekspresi depresi
most_depressive_texts = data_Depression.sort_values(by='similarity_score', ascending=False).head(435)
most_depressive_texts

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/140 [00:00<?, ?it/s]

,statement,status,similarity_score
7051,it s so pointless for me to still be alive my ...,Depression,0.650691
4363,guys i hope you read this i have anything i ne...,Depression,0.636519
14264,i feel empty and tired what even is the point ...,Depression,0.633260
4469,i feel completely exhausted my life isn t goin...,Depression,0.630433
418,i am tired i don t feel anything good i just d...,Depression,0.623807
...,...,...,...
9769,i am not afraid that i have not prepared well ...,Depression,0.479607
10559,i am sad and wore down i am tired of going on ...,Depression,0.479393
13720,i have no friends i spend all week working onl...,Depression,0.479372
2397,ill be here tm and when i take my last breath ...,Depression,0.478915


In [10]:
from tqdm import tqdm

selected_texts = most_depressive_texts['statement'].dropna().tolist()

# Fungsi untuk format teks ke MarianMT
def format_batch_texts(language_code, batch_texts):
    return [">>{}<< {}".format(language_code, text) for text in batch_texts]

# Fungsi untuk menerjemahkan dengan MarianMT
def perform_translation(texts, model, tokenizer, language="id"):
    formatted_texts = format_batch_texts(language, texts)
    inputs = tokenizer(formatted_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Fungsi back translation dengan batas re-translation satu kali jika hasil pertama duplikat
def perform_back_translation(texts, original_language="en", temporary_language="id", batch_size=32):
    back_translated = []
    tmp_translations = []
    
    pbar = tqdm(total=len(texts), desc="Back translation progress")
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Translate to temporary language
        tmp_translated = perform_translation(batch_texts, first_model, first_model_tkn, temporary_language)
        tmp_translations.extend(tmp_translated)
        
        # Translate back to original language
        batch_back_translated = perform_translation(tmp_translated, second_model, second_model_tkn, original_language)
        
        # Cek duplikasi dan lakukan re-translation hanya sekali jika perlu
        final_batch = []
        for j, back_text in enumerate(batch_back_translated):
            if back_text == batch_texts[j] or back_text in back_translated:
                # Lakukan re-translation hanya sekali
                tmp_retranslated = perform_translation([batch_texts[j]], first_model, first_model_tkn, temporary_language)
                back_text_retranslated = perform_translation(tmp_retranslated, second_model, second_model_tkn, original_language)[0]
                
                # Jika hasil berbeda dan belum ada di dataset, gunakan
                if back_text_retranslated != batch_texts[j] and back_text_retranslated not in back_translated:
                    final_batch.append(back_text_retranslated)
                else:
                    final_batch.append(back_text)  # Jika tetap sama, gunakan hasil awal
            else:
                final_batch.append(back_text)
        
        back_translated.extend(final_batch)
        
        # Update progress bar dan tampilkan jumlah teks yang telah diproses
        pbar.update(len(batch_texts))
        print(f"Teks yang sudah diproses: {len(back_translated)} dari {len(texts)}")
    
    pbar.close()
    print(f"Proses back translation selesai! Total teks yang diproses: {len(back_translated)}")
    return back_translated, tmp_translations

# Jalankan back translation dengan batch size 16
back_translated_texts, tmp_translated_texts = perform_back_translation(selected_texts, batch_size=32)

# Fungsi untuk membersihkan teks
def clean_text(text):
    return text.lower()

# Bersihkan teks yang telah diperoleh
cleaned_texts = [clean_text(text) for text in back_translated_texts]

# Buat DataFrame baru dengan hasil augmentasi
df_augmented = pd.DataFrame({
    'original_text': selected_texts,
    'temporary_translation': tmp_translated_texts,
    'statement': cleaned_texts,
    'status': 'Depression'
})

# Tampilkan hasil
df_augmented

Back translation progress:   7%|▋         | 32/435 [00:50<10:35,  1.58s/it]

Teks yang sudah diproses: 32 dari 435


Back translation progress:  15%|█▍        | 64/435 [01:42<09:54,  1.60s/it]

Teks yang sudah diproses: 64 dari 435


Back translation progress:  22%|██▏       | 96/435 [02:13<07:25,  1.31s/it]

...

Back translation progress:  96%|█████████▌| 416/435 [11:14<00:31,  1.66s/it]

Teks yang sudah diproses: 416 dari 435


Back translation progress: 100%|██████████| 435/435 [11:48<00:00,  1.63s/it]

Teks yang sudah diproses: 435 dari 435
Proses back translation selesai! Total teks yang diproses: 435


,original_text,temporary_translation,statement,status
0,it s so pointless for me to still be alive my ...,Tidak ada gunanya bagiku untuk tetap hidup. Ke...,there's no point in me staying alive.,Depression
1,guys i hope you read this i have anything i ne...,pria saya berharap Anda membaca ini saya memil...,i hope you read this. i have nothing that i ne...,Depression
2,i feel empty and tired what even is the point ...,Aku merasa kosong dan lelah bahkan apa gunanya...,i feel empty and tired even what's the point o...,Depression
3,i feel completely exhausted my life isn t goin...,Aku merasa benar-benar lelah hidupku tidak aka...,i feel really tired that my life isn't going a...,Depression
4,i am tired i don t feel anything good i just d...,aku lelah aku tidak merasakan sesuatu yang bai...,i'm tired. i don't feel anything good. i just ...,Depression
...,...,...,...,...
430,i am not afraid that i have not prepared well ...,aku tidak takut bahwa aku tidak siap baik atau...,i'm not afraid that i'm not ready either or i'...,Depression
431,i am sad and wore down i am tired of going on ...,Aku sedih dan lelah aku lelah terus aku benar-...,i'm sad and tired i'm tired of going on i'm re...,Depression
432,i have no friends i spend all week working onl...,Aku tidak punya teman Aku menghabiskan sepanja...,i didn't have any friends i spent all week wor...,Depression
433,ill be here tm and when i take my last breath ...,Sakit berada di sini tm dan ketika saya mengam...,it hurts to be here tm and when i take my last...,Depression


In [11]:
# Mengatur agar teks panjang ditampilkan utuh
pd.set_option('display.max_colwidth', None)

# Menampilkan satu baris teks dari DataFrame untuk ketiga kolom
print(df_augmented.iloc[3])  # Menampilkan seluruh baris pertama

original_text                                i feel completely exhausted my life isn t going anywhere and i ve got nobody to turn to
temporary_translation    Aku merasa benar-benar lelah hidupku tidak akan kemana-mana dan aku tidak punya siapa-siapa untuk berpaling
statement                                  i feel really tired that my life isn't going anywhere and i don't have anyone to turn to.
status                                                                                                                    Depression
Name: 3, dtype: object


In [12]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score

# ==================== BLEU Score Without Smoothing ====================
def calculate_bleu_score_no_smoothing(reference_texts, translated_texts):
    bleu_scores = []
    for ref, trans in zip(reference_texts, translated_texts):
        ref_tokens = ref.split()  # Tokenize reference
        trans_tokens = trans.split()  # Tokenize translation
        bleu_score = sentence_bleu([ref_tokens], trans_tokens)  # BLEU score without smoothing
        bleu_scores.append(bleu_score)
    return bleu_scores

bleu_scores_no_smoothing = calculate_bleu_score_no_smoothing(selected_texts, back_translated_texts)

# ==================== BERT Score ====================
def calculate_bert_score(reference_texts, translated_texts, lang="en"):
    P, R, F1 = score(translated_texts, reference_texts, lang=lang)
    return P, R, F1

P, R, F1 = calculate_bert_score(selected_texts, back_translated_texts)

# Add scores to the DataFrame
df_augmented['BLEU_Score'] = bleu_scores_no_smoothing
df_augmented['BERT_F1'] = F1.tolist()

# Calculate average scores
average_bleu = df_augmented['BLEU_Score'].mean()
average_bert_f1 = df_augmented['BERT_F1'].mean()

# Display average scores
print(f"\nAverage BLEU Score: {average_bleu:.4f}")
print(f"Average BERT F1 Score: {average_bert_f1:.4f}")

df_augmented

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Average BLEU Score: 0.2196
Average BERT F1 Score: 0.9075


,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,it s so pointless for me to still be alive my life is worthless why am i still here,Tidak ada gunanya bagiku untuk tetap hidup. Kenapa aku masih di sini?,there's no point in me staying alive.,Depression,0.110719,0.898112
1,guys i hope you read this i have anything i need financially but i feel so empty inside i am years old guy i lost the emotional attachment to anybody even my parents i do not care if they live or die just so you can get the point how numb emotionally i becameampxbi do not care about anything ifeel empty inside i feel pain of emptiness in my heart every second of the day i do not know what to do with life in my opinion i see life as meaningless honestlyampxbmoney just adds up my pain as i was thinking it is the sauce for me to become happy i feel empty inside,pria saya berharap Anda membaca ini saya memiliki apa-apa yang saya butuhkan secara finansial tapi saya merasa begitu kosong di dalam saya bertahun-tahun orang tua saya kehilangan ikatan emosional kepada siapa pun bahkan orang tua saya saya tidak peduli apakah mereka hidup atau mati hanya sehingga Anda bisa mendapatkan titik bagaimana mati rasa emosional saya menjadiampxbi tidak peduli tentang apa pun saya merasa kosong di dalam hati saya merasa rasa kekosongan dalam hati saya setiap detik dari hari saya tidak tahu apa yang harus dilakukan dengan kehidupan dalam pendapat saya saya saya saya melihat hidup sebagai jujur tidak berarti sebagai ampxbil money hanya menambahkan rasa sakit saya berpikir itu adalah saus bagi saya untuk menjadi bahagia saya merasa kosong di dalam saya merasa kosong di dalam saya merasa kosong,i hope you read this. i have nothing that i need financially but i feel so empty inside me. all these years my parents lost their emotional ties to anyone.,Depression,0.009974,0.874906
2,i feel empty and tired what even is the point of living if we are all going to die anyway i am just trying to keep all together for the people who loves me cuz i know they are going to be devastated when i kms but shit it is hard and exhaustingi just want to take a rest forever like die peacefully in my sleep and be forgotten by everyone existing is exhausting,Aku merasa kosong dan lelah bahkan apa gunanya hidup jika kita semua akan mati bagaimanapun aku hanya mencoba untuk menjaga semua bersama-sama untuk orang-orang yang mencintaiku karena aku tahu mereka akan hancur ketika i kms tapi kotoran itu sulit dan melelahkani hanya ingin beristirahat selamanya seperti mati damai dalam tidur dan dilupakan oleh semua orang yang ada melelahkan,i feel empty and tired even what's the point of living if we're all going to die anyway i'm just trying to keep it all together for the people who love me because i know they'll be destroyed when i kms but that shit is hard and exhausting just want to rest forever like death in peace and be forgotten by everyone who exists tiring,Depression,0.349616,0.942244
3,i feel completely exhausted my life isn t going anywhere and i ve got nobody to turn to,Aku merasa benar-benar lelah hidupku tidak akan kemana-mana dan aku tidak punya siapa-siapa untuk berpaling,i feel really tired that my life isn't going anywhere and i don't have anyone to turn to.,Depression,0.284333,0.938843
4,i am tired i don t feel anything good i just don t want to exist anymore,aku lelah aku tidak merasakan sesuatu yang baik aku hanya tidak ingin ada lagi,i'm tired. i don't feel anything good. i just don't want any more.,Depression,0.294174,0.932004
...,...,...,...,...,...,...
430,i am not afraid that i have not prepared well or i will fail i am not even thinking about anything related to studies education future or even myself i do not know what is happening with me i have no clue why the hell i am even sad i am feeling very discomforted i have not faced any trauma but still i am in pain maybe i am depressed but how the hell someone can be depressed without any reasoni just do not know wha

In [13]:
# Save the augmented DataFrame
df_augmented.to_csv('/kaggle/working/augmented_Depression_BackT_Fixx.csv', index=False)

In [14]:
# Menghapus semua tanda baca atau karakter non-alfabet dari kolom statement
df_augmented['statement'] = df_augmented['statement'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Menampilkan hasil setelah penghapusan tanda baca
df_augmented

,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,it s so pointless for me to still be alive my life is worthless why am i still here,Tidak ada gunanya bagiku untuk tetap hidup. Kenapa aku masih di sini?,theres no point in me staying alive,Depression,0.110719,0.898112
1,guys i hope you read this i have anything i need financially but i feel so empty inside i am years old guy i lost the emotional attachment to anybody even my parents i do not care if they live or die just so you can get the point how numb emotionally i becameampxbi do not care about anything ifeel empty inside i feel pain of emptiness in my heart every second of the day i do not know what to do with life in my opinion i see life as meaningless honestlyampxbmoney just adds up my pain as i was thinking it is the sauce for me to become happy i feel empty inside,pria saya berharap Anda membaca ini saya memiliki apa-apa yang saya butuhkan secara finansial tapi saya merasa begitu kosong di dalam saya bertahun-tahun orang tua saya kehilangan ikatan emosional kepada siapa pun bahkan orang tua saya saya tidak peduli apakah mereka hidup atau mati hanya sehingga Anda bisa mendapatkan titik bagaimana mati rasa emosional saya menjadiampxbi tidak peduli tentang apa pun saya merasa kosong di dalam hati saya merasa rasa kekosongan dalam hati saya setiap detik dari hari saya tidak tahu apa yang harus dilakukan dengan kehidupan dalam pendapat saya saya saya saya melihat hidup sebagai jujur tidak berarti sebagai ampxbil money hanya menambahkan rasa sakit saya berpikir itu adalah saus bagi saya untuk menjadi bahagia saya merasa kosong di dalam saya merasa kosong di dalam saya merasa kosong,i hope you read this i have nothing that i need financially but i feel so empty inside me all these years my parents lost their emotional ties to anyone,Depression,0.009974,0.874906
2,i feel empty and tired what even is the point of living if we are all going to die anyway i am just trying to keep all together for the people who loves me cuz i know they are going to be devastated when i kms but shit it is hard and exhaustingi just want to take a rest forever like die peacefully in my sleep and be forgotten by everyone existing is exhausting,Aku merasa kosong dan lelah bahkan apa gunanya hidup jika kita semua akan mati bagaimanapun aku hanya mencoba untuk menjaga semua bersama-sama untuk orang-orang yang mencintaiku karena aku tahu mereka akan hancur ketika i kms tapi kotoran itu sulit dan melelahkani hanya ingin beristirahat selamanya seperti mati damai dalam tidur dan dilupakan oleh semua orang yang ada melelahkan,i feel empty and tired even whats the point of living if were all going to die anyway im just trying to keep it all together for the people who love me because i know theyll be destroyed when i kms but that shit is hard and exhausting just want to rest forever like death in peace and be forgotten by everyone who exists tiring,Depression,0.349616,0.942244
3,i feel completely exhausted my life isn t going anywhere and i ve got nobody to turn to,Aku merasa benar-benar lelah hidupku tidak akan kemana-mana dan aku tidak punya siapa-siapa untuk berpaling,i feel really tired that my life isnt going anywhere and i dont have anyone to turn to,Depression,0.284333,0.938843
4,i am tired i don t feel anything good i just don t want to exist anymore,aku lelah aku tidak merasakan sesuatu yang baik aku hanya tidak ingin ada lagi,im tired i dont feel anything good i just dont want any more,Depression,0.294174,0.932004
...,...,...,...,...,...,...
430,i am not afraid that i have not prepared well or i will fail i am not even thinking about anything related to studies education future or even myself i do not know what is happening with me i have no clue why the hell i am even sad i am feeling very discomforted i have not faced any trauma but still i am in pain maybe i am depressed but how the hell someone can be depressed without any reasoni just do not know what to do where to g

In [15]:
# Menyisakan hanya kolom 'statement' dan 'status'
df_depresi = df_augmented.loc[:, ['statement', 'status']]

# Menampilkan DataFrame hasil
df_depresi

,statement,status
0,theres no point in me staying alive,Depression
1,i hope you read this i have nothing that i need financially but i feel so empty inside me all these years my parents lost their emotional ties to anyone,Depression
2,i feel empty and tired even whats the point of living if were all going to die anyway im just trying to keep it all together for the people who love me because i know theyll be destroyed when i kms but that shit is hard and exhausting just want to rest forever like death in peace and be forgotten by everyone who exists tiring,Depression
3,i feel really tired that my life isnt going anywhere and i dont have anyone to turn to,Depression
4,im tired i dont feel anything good i just dont want any more,Depression
...,...,...
430,im not afraid that im not ready either or im going to fail im not even thinking about anything related to future education or even myself i dont know whats going to happen with me,Depression
431,im sad and tired im tired of going on im really sad clinical depression is mf sleep doesnt help the sunlight dont help drink it doesnt seem to do much longer well i wish i could buy therapy or drugs or just something im really fking sad right now,Depression
432,i didnt have any friends i spent all week working just looking forward to when i could get a smoke house together and go to my ex bed out there making friends and having adventures while i just jerked into a bikini picture he posted on an instagram i felt like a loser i was such a fucking mess,Depression
433,it hurts to be here tm and when i take my last breath theres no fucking giving up anyway and life will always be a fucking lie youre just afraid to die yeah lol theres all there for me just wipe its no fucking good,Depression


In [16]:
# Save the augmented DataFrame
df_depresi.to_csv('/kaggle/working/dataBT_Depression_aftersemhas.csv', index=False)

## Anxiety ##

In [17]:
import torch
# Pilih GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model names for translation
first_model_name = 'Helsinki-NLP/opus-mt-en-id'
second_model_name = 'Helsinki-NLP/opus-mt-id-en'

# Load the tokenizers
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained models and move them to GPU
first_model = MarianMTModel.from_pretrained(first_model_name).to(device)
second_model = MarianMTModel.from_pretrained(second_model_name).to(device)

In [18]:
# Filter the data for 'Depression' status
data_anxiety = data[data['status'] == 'Anxiety']
original_texts = data_anxiety['statement'].tolist()
data_anxiety

,statement,status
6,fear of getting schizophrenia or any mental illness so my psychologist told me i have gad with obsessive thoughts that are health related im very scared of getting schizophrenia yesterday i had a very vivid dream of two people at my school dying and it felt so real in the dream i passed by a house in a car and two bodies were there and i was crying hard in the dream i woke up all scared and i heard a symptom of getting schizophrenia is believing your dreams or something i havent looked up my symptoms in months and i never will i know all of this is irrational but im scared my psychologist told me that its just anxiety and that i dont have schizophrenia and that im not developing it he said hes seen many patients with that illness and im not like them sometimes i have paranoid thoughts when im in my mind but i know theyre irrational,Anxiety
11,left side of face slightly numb yesterday during the afternoon i took a nap when i woke up my left side of my face felt slightly numb to describe it its essentially feels like when you go outside into cold air the slightly numbness you feel i till can feel things on my face and of course im not showing any other symptoms first thing my anxiety goes to is stroke but no other symptoms have popped up im having some crazy health anxiety over this im not really looking for advice mostly posting this because of my anxiety and seeing if others have gone through something similar to this,Anxiety
20,numbness sensation on chest around heart after waking up once in a while please does anyone else experience this so this is seriously at risk of ruining my day after doing so well for so long so ill try keep it briefi read an article about snoring and sleep apnoea last night according to everyone whos ever slept in a room with me i do snore heavily but dont have sleep apnoeabut this article i read said that even snoring is bad for you and i got freaked out i wake up sometimes with dead or asleep arms making me think theres something seriously wrong with my heart and circulation though my fiance convinced me this is just cause i move my arms to weird positions and the circulation gets cut off temporarilybut something my fiance cant talk me down from is the fact that sometimes i wake up with red indentsmarks all over my body along with like a numbness or asleep feeling around my lower chest on the left side stretching around to my side the thing is i can only ever remember this happening on the left side ie where the heart is but im always very groggy when this happens so maybe it does happen on the right and i dont take notice i only bring it up because it happened recently the sensation goes away within minutes or sohas anyone else ever experienced this it only ever happens when i am in very deep sleep having very vivid dreams and then wake up suddenly,Anxiety
32,how do i cope actually so glad i came across this sub after checking reddit for similar symptoms year old male and i go to the doctors a few times a month with a new issue my doctors laughs when he calls my name out and sees me walk down we do have a good relationship now tho i think he enjoys seeing me as it lightens his mood recently started trt after symptoms of low t got really bad i just saw a urologist today about a lump in my scrotum harmless cyst apparently i saw my doctors recently about a swollen lymph node in my neck ultrasound booked for next week i started to feel better about it until my mum just saw me feeling it and said my uncle had the same thing and its leukemia apparently it wont effect him till his old but what the hell straight down the rabbit hole googling away heart pumping i bet i have leukemia im certain i dont know what else it could bei have no symptoms of why it could be swollen and the worst thing is i dont really know how long its been like that for i just felt it while rubbing my neck one day about the size of a cashewanyone had a swollen lymph node on there right side that turned out not to 

In [19]:
from tqdm import tqdm
# Pilih teks secara acak
num_texts_to_add = 2164
selected_texts = random.sample(original_texts, num_texts_to_add)

# Fungsi untuk format teks ke MarianMT
def format_batch_texts(language_code, batch_texts):
    return [">>{}<< {}".format(language_code, text) for text in batch_texts]

# Fungsi untuk menerjemahkan dengan MarianMT
def perform_translation(texts, model, tokenizer, language="id"):
    formatted_texts = format_batch_texts(language, texts)
    inputs = tokenizer(formatted_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Fungsi back translation dengan batas re-translation satu kali jika hasil pertama duplikat
def perform_back_translation(texts, original_language="en", temporary_language="id", batch_size=32):
    back_translated = []
    tmp_translations = []
    
    pbar = tqdm(total=len(texts), desc="Back translation progress", dynamic_ncols=True)
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Translate to temporary language
        tmp_translated = perform_translation(batch_texts, first_model, first_model_tkn, temporary_language)
        tmp_translations.extend(tmp_translated)
        
        # Translate back to original language
        batch_back_translated = perform_translation(tmp_translated, second_model, second_model_tkn, original_language)
        
        # Cek duplikasi dan lakukan re-translation hanya sekali jika perlu
        final_batch = []
        for j, back_text in enumerate(batch_back_translated):
            if back_text == batch_texts[j] or back_text in back_translated:
                # Lakukan re-translation hanya sekali
                tmp_retranslated = perform_translation([batch_texts[j]], first_model, first_model_tkn, temporary_language)
                back_text_retranslated = perform_translation(tmp_retranslated, second_model, second_model_tkn, original_language)[0]
                
                # Jika hasil berbeda dan belum ada di dataset, gunakan
                if back_text_retranslated != batch_texts[j] and back_text_retranslated not in back_translated:
                    final_batch.append(back_text_retranslated)
                else:
                    final_batch.append(back_text)  # Jika tetap sama, gunakan hasil awal
            else:
                final_batch.append(back_text)
        
        back_translated.extend(final_batch)
        
        # Update progress bar
        pbar.update(len(batch_texts))
        pbar.set_description(f"Back translation progress: {len(back_translated)}/{len(texts)} teks diproses")

    pbar.close()
    print(f"\nProses back translation selesai! Total teks yang diproses: {len(back_translated)}")
    return back_translated, tmp_translations

# Jalankan back translation dengan batch size 32
back_translated_texts, tmp_translated_texts = perform_back_translation(selected_texts, batch_size=32)

# Fungsi untuk membersihkan teks
def clean_text(text):
    return text.lower()

# Bersihkan teks yang telah diperoleh
cleaned_texts = [clean_text(text) for text in back_translated_texts]

# Buat DataFrame baru dengan hasil augmentasi
df_augmented = pd.DataFrame({
    'original_text': selected_texts,
    'temporary_translation': tmp_translated_texts,
    'statement': cleaned_texts,
    'status': 'Anxiety'
})

# Tampilkan hasil
df_augmented

Back translation progress: 2164/2164 teks diproses: 100%|██████████| 2164/2164 [1:03:09<00:00,  1.75s/it]


Proses back translation selesai! Total teks yang diproses: 2164


,original_text,temporary_translation,statement,status
0,i had no idea so many people were like me ever since i was i had panic attacks from thinking i was having a stroke or heart attacks or cancer etc first was walking home from school started feeling very thirsty and hot then dizzy called thinking i was dying having a heat stroke doc told me i had a panic attack now im and been in and out of work due to my panic attacks or just the anxiety of having panic attacks im super depressed and have no drive to be the best me anymore i feel stuck and like ill never get better pot works wonders but unfortunately i dont live in a legal state and since im looking for a job i cant use it i have to rely on my clonopin which only helps if i catch the panic attack in time i get pains or find something wrong on my body and assume the worst instantly and go full panic mode fucking sucks its ruined my life all i do now is put in job applications and play bfv to keep my mind off my body last income ive had was from a guy i used to work concrete construction with who hires me as a ranch hand ever so often but only for about a week a year my father supports me but i hate relying on him because he barely makes any money doing what he does and we are always short on food because rent and utilities come first i hate doing this to him im fucking i should be out doing my own think not seizuring out thinking im dying times a week,Saya tidak punya ide begitu banyak orang yang seperti saya sejak saya mengalami serangan panik dari berpikir saya mengalami stroke atau serangan jantung atau kanker dll pertama-tama adalah berjalan pulang dari sekolah mulai merasa sangat haus dan panas kemudian pusing berpikir saya mati memiliki dokter stroke panas mengatakan kepada saya memiliki serangan panik sekarang im dan keluar dari pekerjaan karena saya panik serangan atau hanya kekhawatiran memiliki serangan panik im super depresi dan tidak drive untuk menjadi yang terbaik lagi saya merasa seperti sakit tidak pernah mendapatkan panci yang lebih baik tapi sayangnya i dont hidup dalam keadaan hukum negara dan karena saya dapat menggunakan pekerjaan inoping saya hanya membantu jika saya menangkap sakit atau tidak mendapatkan sesuatu yang salah saya merasa lebih baik dan tidak pernah mendapatkan pot yang lebih baik dan tidak pernah bekerja lebih baik dari pekerjaan terakhir saya dan saya tidak memiliki pekerjaan yang membuat saya kehilangan waktu terakhir saya bekerja untuk pekerjaan yang baik dan saya pikir saya harus terus bekerja untuk pekerjaan yang saya lakukan untuk pekerjaan terakhir kali saya tidak memiliki untuk pekerjaan yang saya lakukan untuk saya tidak hanya waktu yang salah waktu yang tepat waktu dan iv bekerja di waktu yang tepat waktu dan saya lakukan dalam waktu terakhir saya harus melakukan pekerjaan yang tepat waktu dan saya lakukan dalam waktu terakhir saya hanya waktu dan saya tidak memiliki waktu terakhir saya tidak memiliki pekerjaan yang tepat untuk bekerja di rumah yang saya tidak memiliki waktu terakhir saya waktu yang baik untuk bekerja di rumah yang buruk dan saya dan saya harus dilakukan saya waktu terakhir saya bekerja di rumah yang saya bekerja di rumah yang baik untuk bekerja di rumah yang saya dan saya hanya waktu terakhir saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik untuk bekerja di rumah yang baik dan saya dan saya waktu terakhir saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik dan saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik untuk bekerja di waktu terakhir saya waktu terakhir saya harus saya harus saya bekerja di rumah dan saya bekerja di rumah yang saya bekerja di rumah dan saya bekerja di waktu terakhir saya bekerja di rumah yang baik untuk bekerja di rumah yang saya dan saya dan saya harus saya harus saya bekerja di waktu terakhir saya bekerja di waktu terakhir saya bekerja di rumah yang saya bekerja di waktu dan saya bekerja di waktu terakhir saya bekerja di waktu terakhir

In [20]:
# Mengatur agar teks panjang ditampilkan utuh
pd.set_option('display.max_colwidth', None)

# Menampilkan satu baris teks dari DataFrame untuk ketiga kolom
print(df_augmented.iloc[321])  # Menampilkan seluruh baris pertama

original_text                                                                                                                            college with crippling anxiety ive tried everything i was withdrawn before because of my lack of attendance months ago due to my anxiety i have started college again after being on zoloft for a while but my anxiety is still just as bad i have gone into college more but i still feel unable to function thinking of all the people around me i have only been in times so far and i cant cope im scared of being withdrawn again just because of my inability to go in sometimes i just want to get on with my life
temporary_translation    perguruan tinggi dengan kecemasan yang melumpuhkan saya telah mencoba segala sesuatu saya ditarik sebelumnya karena kurangnya kehadiran saya bulan lalu karena kecemasan saya saya telah mulai kuliah lagi setelah berada di Zoloft untuk sementara waktu tapi kecemasan saya masih sama buruknya saya pergi ke perguruan tinggi lebih banya

In [21]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score

# ==================== BLEU Score Without Smoothing ====================
def calculate_bleu_score_no_smoothing(reference_texts, translated_texts):
    bleu_scores = []
    for ref, trans in zip(reference_texts, translated_texts):
        ref_tokens = ref.split()  # Tokenize reference
        trans_tokens = trans.split()  # Tokenize translation
        bleu_score = sentence_bleu([ref_tokens], trans_tokens)  # BLEU score without smoothing
        bleu_scores.append(bleu_score)
    return bleu_scores

bleu_scores_no_smoothing = calculate_bleu_score_no_smoothing(selected_texts, back_translated_texts)

# ==================== BERT Score ====================
def calculate_bert_score(reference_texts, translated_texts, lang="en"):
    P, R, F1 = score(translated_texts, reference_texts, lang=lang)
    return P, R, F1

P, R, F1 = calculate_bert_score(selected_texts, back_translated_texts)

# Add scores to the DataFrame
df_augmented['BLEU_Score'] = bleu_scores_no_smoothing
df_augmented['BERT_F1'] = F1.tolist()

# Calculate average scores
average_bleu = df_augmented['BLEU_Score'].mean()
average_bert_f1 = df_augmented['BERT_F1'].mean()

# Display average scores
print(f"\nAverage BLEU Score: {average_bleu:.4f}")
print(f"Average BERT F1 Score: {average_bert_f1:.4f}")

df_augmented

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Average BLEU Score: 0.2386
Average BERT F1 Score: 0.8904


,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,i had no idea so many people were like me ever since i was i had panic attacks from thinking i was having a stroke or heart attacks or cancer etc first was walking home from school started feeling very thirsty and hot then dizzy called thinking i was dying having a heat stroke doc told me i had a panic attack now im and been in and out of work due to my panic attacks or just the anxiety of having panic attacks im super depressed and have no drive to be the best me anymore i feel stuck and like ill never get better pot works wonders but unfortunately i dont live in a legal state and since im looking for a job i cant use it i have to rely on my clonopin which only helps if i catch the panic attack in time i get pains or find something wrong on my body and assume the worst instantly and go full panic mode fucking sucks its ruined my life all i do now is put in job applications and play bfv to keep my mind off my body last income ive had was from a guy i used to work concrete construction with who hires me as a ranch hand ever so often but only for about a week a year my father supports me but i hate relying on him because he barely makes any money doing what he does and we are always short on food because rent and utilities come first i hate doing this to him im fucking i should be out doing my own think not seizuring out thinking im dying times a week,Saya tidak punya ide begitu banyak orang yang seperti saya sejak saya mengalami serangan panik dari berpikir saya mengalami stroke atau serangan jantung atau kanker dll pertama-tama adalah berjalan pulang dari sekolah mulai merasa sangat haus dan panas kemudian pusing berpikir saya mati memiliki dokter stroke panas mengatakan kepada saya memiliki serangan panik sekarang im dan keluar dari pekerjaan karena saya panik serangan atau hanya kekhawatiran memiliki serangan panik im super depresi dan tidak drive untuk menjadi yang terbaik lagi saya merasa seperti sakit tidak pernah mendapatkan panci yang lebih baik tapi sayangnya i dont hidup dalam keadaan hukum negara dan karena saya dapat menggunakan pekerjaan inoping saya hanya membantu jika saya menangkap sakit atau tidak mendapatkan sesuatu yang salah saya merasa lebih baik dan tidak pernah mendapatkan pot yang lebih baik dan tidak pernah bekerja lebih baik dari pekerjaan terakhir saya dan saya tidak memiliki pekerjaan yang membuat saya kehilangan waktu terakhir saya bekerja untuk pekerjaan yang baik dan saya pikir saya harus terus bekerja untuk pekerjaan yang saya lakukan untuk pekerjaan terakhir kali saya tidak memiliki untuk pekerjaan yang saya lakukan untuk saya tidak hanya waktu yang salah waktu yang tepat waktu dan iv bekerja di waktu yang tepat waktu dan saya lakukan dalam waktu terakhir saya harus melakukan pekerjaan yang tepat waktu dan saya lakukan dalam waktu terakhir saya hanya waktu dan saya tidak memiliki waktu terakhir saya tidak memiliki pekerjaan yang tepat untuk bekerja di rumah yang saya tidak memiliki waktu terakhir saya waktu yang baik untuk bekerja di rumah yang buruk dan saya dan saya harus dilakukan saya waktu terakhir saya bekerja di rumah yang saya bekerja di rumah yang baik untuk bekerja di rumah yang saya dan saya hanya waktu terakhir saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik untuk bekerja di rumah yang baik dan saya dan saya waktu terakhir saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik dan saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik untuk bekerja di waktu terakhir saya waktu terakhir saya harus saya harus saya bekerja di rumah dan saya bekerja di rumah yang saya bekerja di rumah dan saya bekerja di waktu terakhir saya bekerja di rumah yang baik untuk bekerja di rumah yang saya dan saya dan saya harus saya harus saya bekerja di waktu terakhir saya bekerja di waktu terakhir saya bekerja di rumah yang saya bekerja di waktu dan saya bekerja di waktu terakhir saya bekerj

In [22]:
# Save the augmented DataFrame
df_augmented.to_csv('/kaggle/working/augmented_Anxiety_aftersemhasBLEU.csv', index=False)

In [23]:
df_augmented=pd.read_csv('/kaggle/working/augmented_Anxiety_aftersemhasBLEU.csv')
df_augmented

,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,i had no idea so many people were like me ever since i was i had panic attacks from thinking i was having a stroke or heart attacks or cancer etc first was walking home from school started feeling very thirsty and hot then dizzy called thinking i was dying having a heat stroke doc told me i had a panic attack now im and been in and out of work due to my panic attacks or just the anxiety of having panic attacks im super depressed and have no drive to be the best me anymore i feel stuck and like ill never get better pot works wonders but unfortunately i dont live in a legal state and since im looking for a job i cant use it i have to rely on my clonopin which only helps if i catch the panic attack in time i get pains or find something wrong on my body and assume the worst instantly and go full panic mode fucking sucks its ruined my life all i do now is put in job applications and play bfv to keep my mind off my body last income ive had was from a guy i used to work concrete construction with who hires me as a ranch hand ever so often but only for about a week a year my father supports me but i hate relying on him because he barely makes any money doing what he does and we are always short on food because rent and utilities come first i hate doing this to him im fucking i should be out doing my own think not seizuring out thinking im dying times a week,Saya tidak punya ide begitu banyak orang yang seperti saya sejak saya mengalami serangan panik dari berpikir saya mengalami stroke atau serangan jantung atau kanker dll pertama-tama adalah berjalan pulang dari sekolah mulai merasa sangat haus dan panas kemudian pusing berpikir saya mati memiliki dokter stroke panas mengatakan kepada saya memiliki serangan panik sekarang im dan keluar dari pekerjaan karena saya panik serangan atau hanya kekhawatiran memiliki serangan panik im super depresi dan tidak drive untuk menjadi yang terbaik lagi saya merasa seperti sakit tidak pernah mendapatkan panci yang lebih baik tapi sayangnya i dont hidup dalam keadaan hukum negara dan karena saya dapat menggunakan pekerjaan inoping saya hanya membantu jika saya menangkap sakit atau tidak mendapatkan sesuatu yang salah saya merasa lebih baik dan tidak pernah mendapatkan pot yang lebih baik dan tidak pernah bekerja lebih baik dari pekerjaan terakhir saya dan saya tidak memiliki pekerjaan yang membuat saya kehilangan waktu terakhir saya bekerja untuk pekerjaan yang baik dan saya pikir saya harus terus bekerja untuk pekerjaan yang saya lakukan untuk pekerjaan terakhir kali saya tidak memiliki untuk pekerjaan yang saya lakukan untuk saya tidak hanya waktu yang salah waktu yang tepat waktu dan iv bekerja di waktu yang tepat waktu dan saya lakukan dalam waktu terakhir saya harus melakukan pekerjaan yang tepat waktu dan saya lakukan dalam waktu terakhir saya hanya waktu dan saya tidak memiliki waktu terakhir saya tidak memiliki pekerjaan yang tepat untuk bekerja di rumah yang saya tidak memiliki waktu terakhir saya waktu yang baik untuk bekerja di rumah yang buruk dan saya dan saya harus dilakukan saya waktu terakhir saya bekerja di rumah yang saya bekerja di rumah yang baik untuk bekerja di rumah yang saya dan saya hanya waktu terakhir saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik untuk bekerja di rumah yang baik dan saya dan saya waktu terakhir saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik dan saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik untuk bekerja di waktu terakhir saya waktu terakhir saya harus saya harus saya bekerja di rumah dan saya bekerja di rumah yang saya bekerja di rumah dan saya bekerja di waktu terakhir saya bekerja di rumah yang baik untuk bekerja di rumah yang saya dan saya dan saya harus saya harus saya bekerja di waktu terakhir saya bekerja di waktu terakhir saya bekerja di rumah yang saya bekerja di waktu dan saya bekerja di waktu terakhir saya bekerj

In [24]:
# Menghapus semua tanda baca atau karakter non-alfabet dari kolom statement
df_augmented['statement'] = df_augmented['statement'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Menampilkan hasil setelah penghapusan tanda baca
df_augmented

,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,i had no idea so many people were like me ever since i was i had panic attacks from thinking i was having a stroke or heart attacks or cancer etc first was walking home from school started feeling very thirsty and hot then dizzy called thinking i was dying having a heat stroke doc told me i had a panic attack now im and been in and out of work due to my panic attacks or just the anxiety of having panic attacks im super depressed and have no drive to be the best me anymore i feel stuck and like ill never get better pot works wonders but unfortunately i dont live in a legal state and since im looking for a job i cant use it i have to rely on my clonopin which only helps if i catch the panic attack in time i get pains or find something wrong on my body and assume the worst instantly and go full panic mode fucking sucks its ruined my life all i do now is put in job applications and play bfv to keep my mind off my body last income ive had was from a guy i used to work concrete construction with who hires me as a ranch hand ever so often but only for about a week a year my father supports me but i hate relying on him because he barely makes any money doing what he does and we are always short on food because rent and utilities come first i hate doing this to him im fucking i should be out doing my own think not seizuring out thinking im dying times a week,Saya tidak punya ide begitu banyak orang yang seperti saya sejak saya mengalami serangan panik dari berpikir saya mengalami stroke atau serangan jantung atau kanker dll pertama-tama adalah berjalan pulang dari sekolah mulai merasa sangat haus dan panas kemudian pusing berpikir saya mati memiliki dokter stroke panas mengatakan kepada saya memiliki serangan panik sekarang im dan keluar dari pekerjaan karena saya panik serangan atau hanya kekhawatiran memiliki serangan panik im super depresi dan tidak drive untuk menjadi yang terbaik lagi saya merasa seperti sakit tidak pernah mendapatkan panci yang lebih baik tapi sayangnya i dont hidup dalam keadaan hukum negara dan karena saya dapat menggunakan pekerjaan inoping saya hanya membantu jika saya menangkap sakit atau tidak mendapatkan sesuatu yang salah saya merasa lebih baik dan tidak pernah mendapatkan pot yang lebih baik dan tidak pernah bekerja lebih baik dari pekerjaan terakhir saya dan saya tidak memiliki pekerjaan yang membuat saya kehilangan waktu terakhir saya bekerja untuk pekerjaan yang baik dan saya pikir saya harus terus bekerja untuk pekerjaan yang saya lakukan untuk pekerjaan terakhir kali saya tidak memiliki untuk pekerjaan yang saya lakukan untuk saya tidak hanya waktu yang salah waktu yang tepat waktu dan iv bekerja di waktu yang tepat waktu dan saya lakukan dalam waktu terakhir saya harus melakukan pekerjaan yang tepat waktu dan saya lakukan dalam waktu terakhir saya hanya waktu dan saya tidak memiliki waktu terakhir saya tidak memiliki pekerjaan yang tepat untuk bekerja di rumah yang saya tidak memiliki waktu terakhir saya waktu yang baik untuk bekerja di rumah yang buruk dan saya dan saya harus dilakukan saya waktu terakhir saya bekerja di rumah yang saya bekerja di rumah yang baik untuk bekerja di rumah yang saya dan saya hanya waktu terakhir saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik untuk bekerja di rumah yang baik dan saya dan saya waktu terakhir saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik dan saya tidak memiliki waktu terakhir saya bekerja di rumah yang baik untuk bekerja di waktu terakhir saya waktu terakhir saya harus saya harus saya bekerja di rumah dan saya bekerja di rumah yang saya bekerja di rumah dan saya bekerja di waktu terakhir saya bekerja di rumah yang baik untuk bekerja di rumah yang saya dan saya dan saya harus saya harus saya bekerja di waktu terakhir saya bekerja di waktu terakhir saya bekerja di rumah yang saya bekerja di waktu dan saya bekerja di waktu terakhir saya bekerj

In [25]:
# Menyisakan hanya kolom 'statement' dan 'status'
df_depresi = df_augmented.loc[:, ['statement', 'status']]

# Menampilkan DataFrame hasil
df_depresi

,statement,status
0,i dont have such an idea i dont have so many people who like me ive had this panic attack from i think about i had a panic,Anxiety
1,i keep getting twitching around my body for a few days now i keep getting this twitching around my body usually at my feet i cant feel the weakness but im pretty sure it wont be long before my mind starts tricking me into believing im pretty worried it might be something serious like md or als what the chances are,Anxiety
2,the panic attacks have returned what should the ive have done over the years but no panic attacks at yeaaaaaars however the panic attacks have returned in the last weeks and theyre terrible i forgot how theyve read deep breaths and breaths work are not super helpful and i know that to be true but what else do you during one do you only focus on your breathing but thats how you panic under terrible physical symptoms,Anxiety
3,ive had these little pinprickized red or sometimes these brown dots appear in my skin of course the anxious side of my brain has decided that im dying leukemia or some other day though i dont have these other symptoms showing up in that direction whats particularly strange is that these spots appear in certain areas and then go after a few days they appear in my right hand and my arms after a few days later i spend the night with my girlfriend they go around my two days there are these little spots on my right foot and my feet and there seems to be nothing else on my body and there might be something else that might happen like this might see some doctor and then what i did later,Anxiety
4,tonight im restless,Anxiety
...,...,...
2159,i had an anxiety attack during the action scene in the new john wick movie today i decided to watch john wicks latest film after work today so decided to have two energy drinks full of caffeine to keep me up,Anxiety
2160,juni walking in a crowded room everyone felt super awkward just walking into a crowded room or even just as light as a lonely resident where i saw i was walking normally was people looking at me not in a narcissistic but inferior way myself but the way that had been this whole problem my whole life could have been as simple as being in a rather crowded gym finishing sets in a designated area and running all over the room to pick up paper and spray cleaning to erase my station or walking through a car with people in this thing is a different level if there were others out there who were having anxiety or out there,Anxiety
2161,i was afraid about going to chiropractor ve had a neckache and stiffness for months and after mri and emg found there was nothing like herniated disks or tumors i tried to pt and just got worse with periods of light and nausea so desperate to be relieved that i started going to chiropractor that i had always been skeptical as a profession that xroprarayed my spine said my spine was very stiff and i shouldnt have looked at it when i thought my joints were worse than two days after he used a kirovical action on my spine or there was no energy like that looked like that i should have looked up my neck and had some of the neck injury and had some of the bones that i saw a few of those rare bonebones and i saw some of the bones that were more complicated and i saw some of the bonebones and i saw some of the boneabones that had some things that i had some of the bone damage i had a few of the bone injury and i had some of some of the bone injury and i had some things that i had suffered some of the bone injury and i had some of those things that i had a few of those things that i had suffered and i had some of those things that i had suffered,Anxiety
2162,i should see a neurologist of me f and have gad btw gone to a specialist diagnosed me with an occipitial neuralgia i ate a drug and i no longer get this constant headache he did a physical check on me to check the balance of vision and muscle strength and its fine he told me to fix my posture but my muscles convulsion and d

In [26]:
# Save the augmented DataFrame
df_depresi.to_csv('/kaggle/working/dataBT_Anxiety_aftersemhas.csv', index=False)

## Bipolar ##

In [27]:
import torch
# Pilih GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model names for translation
first_model_name = 'Helsinki-NLP/opus-mt-en-id'
second_model_name = 'Helsinki-NLP/opus-mt-id-en'

# Load the tokenizers
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained models and move them to GPU
first_model = MarianMTModel.from_pretrained(first_model_name).to(device)
second_model = MarianMTModel.from_pretrained(second_model_name).to(device)

In [28]:
# Filter the data for 'Depression' status
data_bipolar = data[data['status'] == 'Bipolar']
original_texts = data_bipolar['statement'].tolist()
data_bipolar

,statement,status
3,husband stayed home from work today again hes staying home because the baby woke up way earlier than his normal wake up time and hasnt been sleeping good the last few days i have not been mentally okay i recently started taking lamictal and im hopeful that ill feel better he stayed home because he said im not stable and he wants to take care of the baby today he did this once before and he wont get in trouble at his job or anything he has enough personal days and vacation days but i feel so ashamed im upset with myself,Bipolar
25,is my therapist at fault or am i being paranoid f ive been seeing my m psychiatrist for the past years and everything has been great hes always been a very good professional and never made me feel uncomfortable considering that ive known how he approaches therapy for years now i find it very strange that he has suddenly changed our boundaries and without asking me first thing is that one day instead of a good bye handshake he said im going to hug you today and he did so i didnt mind goodbye hugging is very common in my culture but i dont like that he didnt ask first if that was ok instead of just announcing it and going for it he proceeded with doing this again after every session since then then last time he kissed me on the cheek also common and nonsexual way to say goodbye in my culture but still strange as hes never done it before he also called me once to check on me and ask how i was feeling i had just lost my dad so maybe thats fine but hes also never done that before all these years today i cancelled our next appointment because something came up and he said ok but i have to pay for it anyway which i know some therapists do but mine never does and i cancelled way earlier than the timespan that would require me to pay what is going on here is this an approach im not aware of is something wrongampxbtldr my therapist of two years suddenly crossed some boundaries which hes never done before and im troubled,Bipolar
37,got prescribed lamotrigine mg not sure what its doing pretty went to the psychiatrist early december of last year and they give me different kinds of medication neither seem to be helping i mean i dont get super mad albeit temper was an issue but maybe the police coming kinda made me see sense in getting to upset but still need to sometimes walk out the room and chill out i dont really feel any different other than real tired and hungry they said it would help with my lack of motivation and drive the other medication helps with serotonin levels but again didnt feel anything my mood is real shitty only with my girlfriend soon to be wife ill berate her with my rants and angry issues for no reason any advice found out i have severe depression and a real lack of motivation and drive my mom suffers from bipolar so the doctors assume i have it pretty much just wondering if their is other medication or if i need to up my dose or anything of the soft and advice would be amazing thanks,Bipolar
38,lithium and mood stabilisation hey guys i have been diagnosed with bipolar affectivw disorder and put on lithium for weeks no im on mg and it pretty much has no effect also on latudaantipsychotic month mg and seroquel ir months mg sleep it almost feels like my hypomania cant be stopped with this combo wouldnt a significant amount of my dopamine be blocked at this point how can i be constantly hypomanic for days and be on lithium my friend tried mg and was floored he said he felt extremely dedated and drunk clurring his words etc and sleeping for like hours im lucky to get hours sleep myself i understand im tolerant to seroquel and probably getting used to latuda but this is rediculous ive had serious arguments with rind coworkers and family due to my expansive mindset and irritabilityrages usually at retards at work or stupid people in public iwould like to believe its justified since theyre retards but even that sound slike a hypomanic thing to say anyway send your xperiences to t

In [29]:
from tqdm import tqdm

# Pilih teks secara acak
num_texts_to_add = 1457
selected_texts = random.sample(original_texts, num_texts_to_add)

# Fungsi untuk format teks ke MarianMT
def format_batch_texts(language_code, batch_texts):
    return [">>{}<< {}".format(language_code, text) for text in batch_texts]

# Fungsi untuk menerjemahkan dengan MarianMT
def perform_translation(texts, model, tokenizer, language="id"):
    formatted_texts = format_batch_texts(language, texts)
    inputs = tokenizer(formatted_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Fungsi back translation dengan batas re-translation satu kali jika hasil pertama duplikat
def perform_back_translation(texts, original_language="en", temporary_language="id", batch_size=32):
    back_translated = []
    tmp_translations = []
    
    pbar = tqdm(total=len(texts), desc="Back translation progress", dynamic_ncols=True)
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Translate to temporary language
        tmp_translated = perform_translation(batch_texts, first_model, first_model_tkn, temporary_language)
        tmp_translations.extend(tmp_translated)
        
        # Translate back to original language
        batch_back_translated = perform_translation(tmp_translated, second_model, second_model_tkn, original_language)
        
        # Cek duplikasi dan lakukan re-translation hanya sekali jika perlu
        final_batch = []
        for j, back_text in enumerate(batch_back_translated):
            if back_text == batch_texts[j] or back_text in back_translated:
                # Lakukan re-translation hanya sekali
                tmp_retranslated = perform_translation([batch_texts[j]], first_model, first_model_tkn, temporary_language)
                back_text_retranslated = perform_translation(tmp_retranslated, second_model, second_model_tkn, original_language)[0]
                
                # Jika hasil berbeda dan belum ada di dataset, gunakan
                if back_text_retranslated != batch_texts[j] and back_text_retranslated not in back_translated:
                    final_batch.append(back_text_retranslated)
                else:
                    final_batch.append(back_text)  # Jika tetap sama, gunakan hasil awal
            else:
                final_batch.append(back_text)
        
        back_translated.extend(final_batch)
        
        # Update progress bar
        pbar.update(len(batch_texts))
        pbar.set_description(f"Back translation progress: {len(back_translated)}/{len(texts)} teks diproses")

    pbar.close()
    print(f"\nProses back translation selesai! Total teks yang diproses: {len(back_translated)}")
    return back_translated, tmp_translations

# Jalankan back translation dengan batch size 32
back_translated_texts, tmp_translated_texts = perform_back_translation(selected_texts, batch_size=32)

# Fungsi untuk membersihkan teks
def clean_text(text):
    return text.lower()

# Bersihkan teks yang telah diperoleh
cleaned_texts = [clean_text(text) for text in back_translated_texts]

# Buat DataFrame baru dengan hasil augmentasi
df_augmented = pd.DataFrame({
    'original_text': selected_texts,
    'temporary_translation': tmp_translated_texts,
    'statement': cleaned_texts,
    'status': 'Bipolar'
})

# Tampilkan hasil
df_augmented

Back translation progress: 1457/1457 teks diproses: 100%|██████████| 1457/1457 [44:35<00:00,  1.84s/it]


Proses back translation selesai! Total teks yang diproses: 1457


,original_text,temporary_translation,statement,status
0,woke up am the past two days and being very high energy is this hypomania cant tell if im just being active and productive or im just being straight up hypomania im bipolar and take meds daily i thought with taking all my meds id never feel heightened or lower but i still do feel my moods go up and down even with meds but the moods are usually mild but the past two days ive just been very active do you all feel the same sometimes,Bangun dua hari terakhir dan menjadi energi yang sangat tinggi adalah hypomania ini tidak bisa mengatakan apakah saya hanya menjadi aktif dan produktif atau im hanya menjadi lurus sampai hypomania im bipolar dan mengambil obat-obatan setiap hari saya pikir dengan mengambil semua obat saya id tidak pernah merasa meningkat atau lebih rendah tapi saya masih merasa suasana hati saya naik dan turun bahkan dengan obat-obatan tapi suasana hati biasanya ringan tapi dua hari terakhir ive hanya sangat aktif apakah Anda semua merasa sama kadang-kadang,this hypomania can't tell whether i just became active and productive or im just being straight up hypomania im bipolar and taking drugs every day i think by taking all the drugs i id never felt higher or lower but i still feel my mood goes up and down even with mild medicine but the last two days of ive are very active whether you all feel the same,Bipolar
1,being dragged into the upsidedown every fall i get dragged into the upsidedown that dark damn place from strangerthings im always stuck there till spring i know that sounds like a gross exaggeration but its not as much of a strech as the normies would assume i feel so alone and everything is darker even visually i see more shadows than are really there and the demigorgon is always just around the corner not a real one but a metaphorical one that represents deep anxietythis year i promised myself i wouldnt let it happen again i telling people i needed help but they didnt really seem to get it i started asked for small accommodations in august but i didnt get any of what i needed till late december its too late now im stuck in the upsidedown for another season because people thinks i was just complaining about the winter bluesthe lithium really takes the edge off i dont think ill have to spend next winter in the upsidedown i just wish i was taken seriously,diseret ke atas setiap musim gugur saya mendapatkan diseret ke terbalik bahwa tempat sialan gelap dari hal-hal asing Im selalu terjebak di sana sampai musim semi saya tahu bahwa terdengar seperti berlebihan kotor tapi itu tidak sebanyak banyak dari strech sebagai norma-normies akan berasumsi saya merasa begitu sendirian dan segala sesuatu yang gelap bahkan gelap saya melihat lebih banyak bayangan daripada benar-benar ada dan demigorgon hanya sekitar sudut tidak nyata tapi metafora satu yang mewakili kecemasan dalam tahun ini saya sendiri tidak akan membiarkan hal itu terjadi lagi i memberitahu orang-orang i dibutuhkan tapi mereka tidak benar-benar tampaknya mendapatkan lebih banyak tanya untuk akomodasi kecil tapi i didnt mendapatkan apa yang dibutuhkan sampai terlambat terlalu terlambat saya janji saya sendiri saya sendiri tidak akan membiarkan hal itu terjadi lagi saya benar-benar mengeluhkan musim dingin karena saya benar-benar membutuhkan waktu itu benar-benar tergantung untuk musim dingin karena saya tidak berpikir bahwa saya benar-benar membutuhkan waktu musim dingin yang benar-benar membutuhkan waktu saya benar-benar berpikir saya benar-benar berpikir bahwa saya hanya butuh waktu musim dingin saya hanya berpikir saya hanya beberapa musim dingin yang lebih baik saya hanya berpikir saya hanya berpikir saya hanya beberapa waktu musim dingin saya tidak ingin saya hanya beberapa musim dingin yang saya benar-benar di musim musim dingin saya hanya berpikir saya hanya berpikir saya hanya berpikir saya hanya beberapa musim dingin yang saya hanya berpikir saya hanya beberapa musim dingin yang saya benar-bena

In [30]:
# Mengatur agar teks panjang ditampilkan utuh
pd.set_option('display.max_colwidth', None)

# Menampilkan satu baris teks dari DataFrame untuk ketiga kolom
print(df_augmented.iloc[0])  # Menampilkan seluruh baris pertama

original_text                                                                                                                             woke up am the past two days and being very high energy is this hypomania cant tell if im just being active and productive or im just being straight up hypomania im bipolar and take meds daily i thought with taking all my meds id never feel heightened or lower but i still do feel my moods go up and down even with meds but the moods are usually mild but the past two days ive just been very active do you all feel the same sometimes
temporary_translation    Bangun dua hari terakhir dan menjadi energi yang sangat tinggi adalah hypomania ini tidak bisa mengatakan apakah saya hanya menjadi aktif dan produktif atau im hanya menjadi lurus sampai hypomania im bipolar dan mengambil obat-obatan setiap hari saya pikir dengan mengambil semua obat saya id tidak pernah merasa meningkat atau lebih rendah tapi saya masih merasa suasana hati saya naik dan turun bahkan

In [31]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score

# ==================== BLEU Score Without Smoothing ====================
def calculate_bleu_score_no_smoothing(reference_texts, translated_texts):
    bleu_scores = []
    for ref, trans in zip(reference_texts, translated_texts):
        ref_tokens = ref.split()  # Tokenize reference
        trans_tokens = trans.split()  # Tokenize translation
        bleu_score = sentence_bleu([ref_tokens], trans_tokens)  # BLEU score without smoothing
        bleu_scores.append(bleu_score)
    return bleu_scores

bleu_scores_no_smoothing = calculate_bleu_score_no_smoothing(selected_texts, back_translated_texts)

# ==================== BERT Score ====================
def calculate_bert_score(reference_texts, translated_texts, lang="en"):
    P, R, F1 = score(translated_texts, reference_texts, lang=lang)
    return P, R, F1

P, R, F1 = calculate_bert_score(selected_texts, back_translated_texts)

# Add scores to the DataFrame
df_augmented['BLEU_Score'] = bleu_scores_no_smoothing
df_augmented['BERT_F1'] = F1.tolist()

# Calculate average scores
average_bleu = df_augmented['BLEU_Score'].mean()
average_bert_f1 = df_augmented['BERT_F1'].mean()

# Display average scores
print(f"\nAverage BLEU Score: {average_bleu:.4f}")
print(f"Average BERT F1 Score: {average_bert_f1:.4f}")

df_augmented

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Average BLEU Score: 0.1925
Average BERT F1 Score: 0.8777


,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,woke up am the past two days and being very high energy is this hypomania cant tell if im just being active and productive or im just being straight up hypomania im bipolar and take meds daily i thought with taking all my meds id never feel heightened or lower but i still do feel my moods go up and down even with meds but the moods are usually mild but the past two days ive just been very active do you all feel the same sometimes,Bangun dua hari terakhir dan menjadi energi yang sangat tinggi adalah hypomania ini tidak bisa mengatakan apakah saya hanya menjadi aktif dan produktif atau im hanya menjadi lurus sampai hypomania im bipolar dan mengambil obat-obatan setiap hari saya pikir dengan mengambil semua obat saya id tidak pernah merasa meningkat atau lebih rendah tapi saya masih merasa suasana hati saya naik dan turun bahkan dengan obat-obatan tapi suasana hati biasanya ringan tapi dua hari terakhir ive hanya sangat aktif apakah Anda semua merasa sama kadang-kadang,this hypomania can't tell whether i just became active and productive or im just being straight up hypomania im bipolar and taking drugs every day i think by taking all the drugs i id never felt higher or lower but i still feel my mood goes up and down even with mild medicine but the last two days of ive are very active whether you all feel the same,Bipolar,0.272452,0.922433
1,being dragged into the upsidedown every fall i get dragged into the upsidedown that dark damn place from strangerthings im always stuck there till spring i know that sounds like a gross exaggeration but its not as much of a strech as the normies would assume i feel so alone and everything is darker even visually i see more shadows than are really there and the demigorgon is always just around the corner not a real one but a metaphorical one that represents deep anxietythis year i promised myself i wouldnt let it happen again i telling people i needed help but they didnt really seem to get it i started asked for small accommodations in august but i didnt get any of what i needed till late december its too late now im stuck in the upsidedown for another season because people thinks i was just complaining about the winter bluesthe lithium really takes the edge off i dont think ill have to spend next winter in the upsidedown i just wish i was taken seriously,diseret ke atas setiap musim gugur saya mendapatkan diseret ke terbalik bahwa tempat sialan gelap dari hal-hal asing Im selalu terjebak di sana sampai musim semi saya tahu bahwa terdengar seperti berlebihan kotor tapi itu tidak sebanyak banyak dari strech sebagai norma-normies akan berasumsi saya merasa begitu sendirian dan segala sesuatu yang gelap bahkan gelap saya melihat lebih banyak bayangan daripada benar-benar ada dan demigorgon hanya sekitar sudut tidak nyata tapi metafora satu yang mewakili kecemasan dalam tahun ini saya sendiri tidak akan membiarkan hal itu terjadi lagi i memberitahu orang-orang i dibutuhkan tapi mereka tidak benar-benar tampaknya mendapatkan lebih banyak tanya untuk akomodasi kecil tapi i didnt mendapatkan apa yang dibutuhkan sampai terlambat terlalu terlambat saya janji saya sendiri saya sendiri tidak akan membiarkan hal itu terjadi lagi saya benar-benar mengeluhkan musim dingin karena saya benar-benar membutuhkan waktu itu benar-benar tergantung untuk musim dingin karena saya tidak berpikir bahwa saya benar-benar membutuhkan waktu musim dingin yang benar-benar membutuhkan waktu saya benar-benar berpikir saya benar-benar berpikir bahwa saya hanya butuh waktu musim dingin saya hanya berpikir saya hanya beberapa musim dingin yang lebih baik saya hanya berpikir saya hanya berpikir saya hanya beberapa waktu musim dingin saya tidak ingin saya hanya beberapa musim dingin yang saya benar-benar di musim musim dingin saya hanya berpikir saya hanya berpikir saya hanya berpikir saya hanya beberapa musim dingin yang saya hanya berpikir saya hanya beber

In [32]:
# Save the augmented DataFrame
df_augmented.to_csv('/kaggle/working/augmented_Bipolar_fixx.csv', index=False)

In [33]:
# Menghapus semua tanda baca atau karakter non-alfabet dari kolom statement
df_augmented['statement'] = df_augmented['statement'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Menampilkan hasil setelah penghapusan tanda baca
df_augmented

,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,woke up am the past two days and being very high energy is this hypomania cant tell if im just being active and productive or im just being straight up hypomania im bipolar and take meds daily i thought with taking all my meds id never feel heightened or lower but i still do feel my moods go up and down even with meds but the moods are usually mild but the past two days ive just been very active do you all feel the same sometimes,Bangun dua hari terakhir dan menjadi energi yang sangat tinggi adalah hypomania ini tidak bisa mengatakan apakah saya hanya menjadi aktif dan produktif atau im hanya menjadi lurus sampai hypomania im bipolar dan mengambil obat-obatan setiap hari saya pikir dengan mengambil semua obat saya id tidak pernah merasa meningkat atau lebih rendah tapi saya masih merasa suasana hati saya naik dan turun bahkan dengan obat-obatan tapi suasana hati biasanya ringan tapi dua hari terakhir ive hanya sangat aktif apakah Anda semua merasa sama kadang-kadang,this hypomania cant tell whether i just became active and productive or im just being straight up hypomania im bipolar and taking drugs every day i think by taking all the drugs i id never felt higher or lower but i still feel my mood goes up and down even with mild medicine but the last two days of ive are very active whether you all feel the same,Bipolar,0.272452,0.922433
1,being dragged into the upsidedown every fall i get dragged into the upsidedown that dark damn place from strangerthings im always stuck there till spring i know that sounds like a gross exaggeration but its not as much of a strech as the normies would assume i feel so alone and everything is darker even visually i see more shadows than are really there and the demigorgon is always just around the corner not a real one but a metaphorical one that represents deep anxietythis year i promised myself i wouldnt let it happen again i telling people i needed help but they didnt really seem to get it i started asked for small accommodations in august but i didnt get any of what i needed till late december its too late now im stuck in the upsidedown for another season because people thinks i was just complaining about the winter bluesthe lithium really takes the edge off i dont think ill have to spend next winter in the upsidedown i just wish i was taken seriously,diseret ke atas setiap musim gugur saya mendapatkan diseret ke terbalik bahwa tempat sialan gelap dari hal-hal asing Im selalu terjebak di sana sampai musim semi saya tahu bahwa terdengar seperti berlebihan kotor tapi itu tidak sebanyak banyak dari strech sebagai norma-normies akan berasumsi saya merasa begitu sendirian dan segala sesuatu yang gelap bahkan gelap saya melihat lebih banyak bayangan daripada benar-benar ada dan demigorgon hanya sekitar sudut tidak nyata tapi metafora satu yang mewakili kecemasan dalam tahun ini saya sendiri tidak akan membiarkan hal itu terjadi lagi i memberitahu orang-orang i dibutuhkan tapi mereka tidak benar-benar tampaknya mendapatkan lebih banyak tanya untuk akomodasi kecil tapi i didnt mendapatkan apa yang dibutuhkan sampai terlambat terlalu terlambat saya janji saya sendiri saya sendiri tidak akan membiarkan hal itu terjadi lagi saya benar-benar mengeluhkan musim dingin karena saya benar-benar membutuhkan waktu itu benar-benar tergantung untuk musim dingin karena saya tidak berpikir bahwa saya benar-benar membutuhkan waktu musim dingin yang benar-benar membutuhkan waktu saya benar-benar berpikir saya benar-benar berpikir bahwa saya hanya butuh waktu musim dingin saya hanya berpikir saya hanya beberapa musim dingin yang lebih baik saya hanya berpikir saya hanya berpikir saya hanya beberapa waktu musim dingin saya tidak ingin saya hanya beberapa musim dingin yang saya benar-benar di musim musim dingin saya hanya berpikir saya hanya berpikir saya hanya berpikir saya hanya beberapa musim dingin yang saya hanya berpikir saya hanya bebera

In [34]:
# Menyisakan hanya kolom 'statement' dan 'status'
df_bipolarrr = df_augmented.loc[:, ['statement', 'status']]

# Menampilkan DataFrame hasil
df_bipolarrr

,statement,status
0,this hypomania cant tell whether i just became active and productive or im just being straight up hypomania im bipolar and taking drugs every day i think by taking all the drugs i id never felt higher or lower but i still feel my mood goes up and down even with mild medicine but the last two days of ive are very active whether you all feel the same,Bipolar
1,i get dragged over every fall i get dragged into that dark damn place of things im always stuck there until spring i know that sounds like gross excess but it doesnt sound like a dirty metaphor but it doesnt allow as much of a strech as the normies would assume im feeling so alone and everything thats dark even dark i saw more shadows than really there and for the sake of ogorgon just around the corner of one but it represents a genuine metaphor in this year i wont let it happen im not let it happen anymore i but they dont really seem to get a lot more than likely to get a few questions for the dark and i didnt get anything too late i promise myself i wont let it happen anymore because im really complain about the winter,Bipolar
2,i wanted to cut a layer in my hair like i always do but this time i screwed up so much that i felt so stupid and sad my hair was long hip and a big piece in front of me wasnt even a long chin like a bob with long hair hip behind my men i loved my hair everybody loved my hair and now i feel so embarrassed because everybody asked me what happened i was so sad i kept googling how long to grow but it was so unbalanced and that it bothered me so much in the first place because it was a bit unbalanced and people knew what was going on,Bipolar
3,i got a job interview from my dream i apologized most of the time i only vented myself here naked with meive has been unemployed since last june since a couple of weeks in the hospital,Bipolar
4,youre going to take treatment for the rest of your life of your life youre going to take a treatment for the rest of your life of your life youll have been treated for the rest of your life for the rest of your life of your of me youll have been treated for the time of your time of your life where ive been diagnosed youll have been diagnosed youll have been diagnosed for the rest for the rest of the rest of your life youll take a treatment for the rest of your rest of your life you will ill take the treatment for the last of a second part of the time of the time of the human part of human life where ill be diagnosed of the rest of your life where ill be diagnosed of the rest of the rest of the rest of the rest of the rest of the rest of your life of the rest of the rest of your life for the rest of your life for the rest of the rest of the rest of the rest of your life for the rest of your life for the rest of the rest of the rest of the rest of the rest of the rest of your life you spent of your life for the rest of the rest of your life you spent of your life you spent of the rest of the rest of the rest of the rest of your life you spent of your life you spent of your life you spent for the rest of the rest of the rest of the rest of your life you spent of your life you spent for the rest of your life you spent for the rest of your life you spent of your life you spent of your life you spent for the life you spent of your life you spent of your life you spent of your life you spent of your life you spent you spent you spent of of of of of your life you spent for the life you spent you spent you spent you spent for the life you spent you spent you spent you spent you spent for the life you spent for the life you spent for the life you spent you spent of of of of of of of of of of of your life you spent you spent you spent for the life you spent of of of of of of your life you spent you spent you spent of your life you spent of of your life you spent you spent you spent you spent you spent you spent you spent you spent you spent you spent you spent you spent of your life you spent you spent you spent

In [35]:
# Save the augmented DataFrame
df_bipolarrr.to_csv('/kaggle/working/dataBT_Bipolar_fixx.csv', index=False)

## Stress ##

In [36]:
import torch
# Pilih GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model names for translation
first_model_name = 'Helsinki-NLP/opus-mt-en-id'
second_model_name = 'Helsinki-NLP/opus-mt-id-en'

# Load the tokenizers
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained models and move them to GPU
first_model = MarianMTModel.from_pretrained(first_model_name).to(device)
second_model = MarianMTModel.from_pretrained(second_model_name).to(device)

In [37]:
# Filter the data for 'Depression' status
data_bipolar = data[data['status'] == 'Stress']
original_texts = data_bipolar['statement'].tolist()
data_bipolar

,statement,status
22,and then proceeded to show me the video of the dumb girlfriend struggling to answer the question needless to say i was pissed i said i couldnt believe he would think it was possible that i might be that stupid he insisted that he only asked me as a joke and then i got angrier because in what world is questioning my ability to handle basic logic funny he continued to go on the offensive insisting that he had been kidding that of course he doesnt think im stupid at all and how could i even think he would think that,Stress
23,im not stressed but my body says otherwise im recently graduated from college doing my internship in nursing ive always thought my stress levels are ok but since for years ive noticed that cant seem to beat fungus skin and vaginal even though i rarely get sick i have irritable bowel syndrome stress related nothing inflammatory my hair falls excessively u think im stressed idk,Stress
30,wanna do a stress quiz hello im doing an assignment for my school about stress and i need reposnses this survey is about the stressful situations many face throughout their lives although this survey may not be accurate or realistic all i need are responses and no worries this is anonymous except for your age dhere is the link to the google form if you guys want to fill out the survey gt,Stress
33,it was terrible i got pregnant again after he forcibly took my birth control away my nd son was born in chile i didnt realize once i had my child in chile i couldnt leave with my son without the fathers permission i think that was part of his plan i lived years of brutal hell i finally made it back home in when one of the police officers who were called after he beat me luckily was married to a woman who worked at the american embassy and they got me home within weeks with both my sons,Stress
49,i keep alternating between crying laughing and freaking out im also really upset right now and my boyfriend isnt making me feel better he keeps saying things like this is all your fault and no one is making you think these things and to be patient with things i feel like this is going to kill me,Stress
...,...,...
14888,wife is out of town and im juggling a stressful job and two kids on my own fuck this is hard idk just venting herei manage a software techniciansupport team its stressful on the best of days and i work hours a week well we just lost one of our best guys and our engineers pushed a new release of the software and as fucking always qa dropped the ball and it fucked production up and so caused a bunch of extra work for my team meto top it off this coincided with my phone suddenly breaking screen stopped working can hear calls come in but cant answer my wife is out of town and im juggling a and year oldi feel like im going to melt cry die,Stress
14899,i am just sick of this being my daily life between the derealization and the hypervigilance and paranoia even on good days i still dont feel like a person and thats not even touching the flashbacks and nightmares i dont know what to do anymore i cant find a shrink who i trust or who seems equipped to help with the particular symptoms i have and that seems like my only option what do i do,Stress
14908,im not against working my ass off im just getting down right now no responses to applications can do that im in a state that doesnt even have a real art school i have no funds whatsoever no family to help i dont know where to go from here like how do you even apply for scholarships how do i not starve to death if i can get to art school,Stress
14912,kicked me out of the house and was going to make me leave in the middle of the night multiple times while insulting me and hitting me im sorry to even post this here but i need a little support right now he said this to me two nights ago and came into our guest bedroom where i was sleeping the next day and said sorry kind of i tried to talk to him about it and he said he was under a lot of stress and that he doesnt need my

In [38]:
from tqdm import tqdm

# Pilih teks secara acak
num_texts_to_add = 1401
selected_texts = random.sample(original_texts, num_texts_to_add)

# Fungsi untuk format teks ke MarianMT
def format_batch_texts(language_code, batch_texts):
    return [">>{}<< {}".format(language_code, text) for text in batch_texts]

# Fungsi untuk menerjemahkan dengan MarianMT
def perform_translation(texts, model, tokenizer, language="id"):
    formatted_texts = format_batch_texts(language, texts)
    inputs = tokenizer(formatted_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Fungsi back translation dengan batas re-translation satu kali jika hasil pertama duplikat
def perform_back_translation(texts, original_language="en", temporary_language="id", batch_size=32):
    back_translated = []
    tmp_translations = []
    
    pbar = tqdm(total=len(texts), desc="Back translation progress", dynamic_ncols=True)
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Translate to temporary language
        tmp_translated = perform_translation(batch_texts, first_model, first_model_tkn, temporary_language)
        tmp_translations.extend(tmp_translated)
        
        # Translate back to original language
        batch_back_translated = perform_translation(tmp_translated, second_model, second_model_tkn, original_language)
        
        # Cek duplikasi dan lakukan re-translation hanya sekali jika perlu
        final_batch = []
        for j, back_text in enumerate(batch_back_translated):
            if back_text == batch_texts[j] or back_text in back_translated:
                # Lakukan re-translation hanya sekali
                tmp_retranslated = perform_translation([batch_texts[j]], first_model, first_model_tkn, temporary_language)
                back_text_retranslated = perform_translation(tmp_retranslated, second_model, second_model_tkn, original_language)[0]
                
                # Jika hasil berbeda dan belum ada di dataset, gunakan
                if back_text_retranslated != batch_texts[j] and back_text_retranslated not in back_translated:
                    final_batch.append(back_text_retranslated)
                else:
                    final_batch.append(back_text)  # Jika tetap sama, gunakan hasil awal
            else:
                final_batch.append(back_text)
        
        back_translated.extend(final_batch)
        
        # Update progress bar
        pbar.update(len(batch_texts))
        pbar.set_description(f"Back translation progress: {len(back_translated)}/{len(texts)} teks diproses")

    pbar.close()
    print(f"\nProses back translation selesai! Total teks yang diproses: {len(back_translated)}")
    return back_translated, tmp_translations

# Jalankan back translation dengan batch size 32
back_translated_texts, tmp_translated_texts = perform_back_translation(selected_texts, batch_size=32)

# Fungsi untuk membersihkan teks
def clean_text(text):
    return text.lower()

# Bersihkan teks yang telah diperoleh
cleaned_texts = [clean_text(text) for text in back_translated_texts]

# Buat DataFrame baru dengan hasil augmentasi
df_augmented = pd.DataFrame({
    'original_text': selected_texts,
    'temporary_translation': tmp_translated_texts,
    'statement': cleaned_texts,
    'status': 'Stress'
})

# Tampilkan hasil
df_augmented

Back translation progress: 1401/1401 teks diproses: 100%|██████████| 1401/1401 [34:31<00:00,  1.48s/it]


Proses back translation selesai! Total teks yang diproses: 1401


,original_text,temporary_translation,statement,status
0,hey first time posting here i have been living in my car with my brother and cat since may i am feeling extremely depresseddefeated right now hence why im posting here ill start from the beginning,Hei pertama kali posting di sini saya telah tinggal di mobil saya dengan kakak saya dan kucing karena mungkin saya merasa sangat tertekan dikalahkan sekarang karena itu mengapa saya posting di sini sakit mulai dari awal,"hey, first time posting here i've been living in my car with my brother and cat because maybe i'm feeling very depressed being defeated right now because that's why i'm posting here sick start at the beginning",Stress
1,i only realized this was a stupid idea in my freshman year of college when i started having flashbacks i tried once more to do therapy and almost failed out of school again i buried it all to be dealt with at a more convenient time i have now been working in a job that dont mind for the last year and months i found that november and december were a struggle and because of that i planned to address my stuff by going to therapy starting in,Aku hanya menyadari ini adalah ide bodoh di tahun pertama kuliahku ketika aku mulai mengalami kilas balik aku mencoba sekali lagi untuk melakukan terapi dan hampir gagal keluar dari sekolah lagi aku menguburnya semua untuk ditangani pada waktu yang lebih nyaman aku sekarang telah bekerja dalam pekerjaan yang tidak keberatan untuk tahun terakhir dan bulan aku menemukan bahwa noverber dan dekember adalah perjuangan dan karena itu saya berencana untuk mengatasi barang-barang saya dengan akan mulai terapi,i just realized this was a stupid idea in my first year of college when i started having flashbacks. i tried once again to do therapy and almost failed out of school again. i buried it all for a more comfortable time.,Stress
2,why do i have so many symptoms for stress not being able to manage my anger my motivation going up and down not wanting to do anything doing random things with parts of my body as a tic why have these all come to me when i am not very thankully not depressed,Mengapa saya memiliki begitu banyak gejala stres tidak mampu mengelola kemarahan saya motivasi saya naik dan turun tidak ingin melakukan apa-apa melakukan hal-hal acak dengan bagian-bagian tubuh saya sebagai tic mengapa semua ini datang kepada saya ketika saya tidak sangat berterima kasih tidak tertekan,why do i have so many symptoms of stress not being able to manage my anger my motivation goes up and down don't want to do anything random with my body parts as tic why all this comes to me when i'm not very grateful not to be depressed,Stress
3,i know no break up is fun but you know what i mean without going into too much detail she was unhappy cheated on me broke up with me started dating other guy we also live together its been three weeks and the stress had caused my stomach to twist into knots i cannot untie not really sure where else got with this i just want to stop feeling like this,Aku tahu tidak ada putus yang menyenangkan tapi kau tahu apa yang saya maksud tanpa pergi ke terlalu banyak detail dia tidak bahagia ditipu pada saya mulai berkencan dengan pria lain kami juga hidup bersama sudah tiga minggu dan stres telah menyebabkan perut saya untuk memutar ke knot saya tidak bisa benar-benar yakin di mana lagi punya dengan ini saya hanya ingin berhenti merasa seperti ini,i know there's no breaking up that's fun but you know what i mean without going into too much detail she's unhappy cheated on me start dating another man we also live together already three weeks and stress has caused my stomach to twist into knots i can't really be sure where else i have with this i just want to stop feeling like this,Stress
4,which forced me to acknowledge that i havent been performing to my own expectations enter the anxiety attack sobbing shaking fidgeting and rambling to top it all off i threw away my xanex thinking that i didnt need the safety 

In [39]:
# Mengatur agar teks panjang ditampilkan utuh
pd.set_option('display.max_colwidth', None)

# Menampilkan satu baris teks dari DataFrame untuk ketiga kolom
print(df_augmented.iloc[0])  # Menampilkan seluruh baris pertama

original_text                                   hey first time posting here i have been living in my car with my brother and cat since may i am feeling extremely depresseddefeated right now hence why im posting here ill start from the beginning
temporary_translation    Hei pertama kali posting di sini saya telah tinggal di mobil saya dengan kakak saya dan kucing karena mungkin saya merasa sangat tertekan dikalahkan sekarang karena itu mengapa saya posting di sini sakit mulai dari awal
statement                          hey, first time posting here i've been living in my car with my brother and cat because maybe i'm feeling very depressed being defeated right now because that's why i'm posting here sick start at the beginning
status                                                                                                                                                                                                                                        Stress
Name: 0, dtype: obje

In [40]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score

# ==================== BLEU Score Without Smoothing ====================
def calculate_bleu_score_no_smoothing(reference_texts, translated_texts):
    bleu_scores = []
    for ref, trans in zip(reference_texts, translated_texts):
        ref_tokens = ref.split()  # Tokenize reference
        trans_tokens = trans.split()  # Tokenize translation
        bleu_score = sentence_bleu([ref_tokens], trans_tokens)  # BLEU score without smoothing
        bleu_scores.append(bleu_score)
    return bleu_scores

bleu_scores_no_smoothing = calculate_bleu_score_no_smoothing(selected_texts, back_translated_texts)

# ==================== BERT Score ====================
def calculate_bert_score(reference_texts, translated_texts, lang="en"):
    P, R, F1 = score(translated_texts, reference_texts, lang=lang)
    return P, R, F1

P, R, F1 = calculate_bert_score(selected_texts, back_translated_texts)

# Add scores to the DataFrame
df_augmented['BLEU_Score'] = bleu_scores_no_smoothing
df_augmented['BERT_F1'] = F1.tolist()

# Calculate average scores
average_bleu = df_augmented['BLEU_Score'].mean()
average_bert_f1 = df_augmented['BERT_F1'].mean()

# Display average scores
print(f"\nAverage BLEU Score: {average_bleu:.4f}")
print(f"Average BERT F1 Score: {average_bert_f1:.4f}")

df_augmented

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Average BLEU Score: 0.2562
Average BERT F1 Score: 0.9084


,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,hey first time posting here i have been living in my car with my brother and cat since may i am feeling extremely depresseddefeated right now hence why im posting here ill start from the beginning,Hei pertama kali posting di sini saya telah tinggal di mobil saya dengan kakak saya dan kucing karena mungkin saya merasa sangat tertekan dikalahkan sekarang karena itu mengapa saya posting di sini sakit mulai dari awal,"hey, first time posting here i've been living in my car with my brother and cat because maybe i'm feeling very depressed being defeated right now because that's why i'm posting here sick start at the beginning",Stress,3.632574e-01,0.933453
1,i only realized this was a stupid idea in my freshman year of college when i started having flashbacks i tried once more to do therapy and almost failed out of school again i buried it all to be dealt with at a more convenient time i have now been working in a job that dont mind for the last year and months i found that november and december were a struggle and because of that i planned to address my stuff by going to therapy starting in,Aku hanya menyadari ini adalah ide bodoh di tahun pertama kuliahku ketika aku mulai mengalami kilas balik aku mencoba sekali lagi untuk melakukan terapi dan hampir gagal keluar dari sekolah lagi aku menguburnya semua untuk ditangani pada waktu yang lebih nyaman aku sekarang telah bekerja dalam pekerjaan yang tidak keberatan untuk tahun terakhir dan bulan aku menemukan bahwa noverber dan dekember adalah perjuangan dan karena itu saya berencana untuk mengatasi barang-barang saya dengan akan mulai terapi,i just realized this was a stupid idea in my first year of college when i started having flashbacks. i tried once again to do therapy and almost failed out of school again. i buried it all for a more comfortable time.,Stress,1.601767e-01,0.919187
2,why do i have so many symptoms for stress not being able to manage my anger my motivation going up and down not wanting to do anything doing random things with parts of my body as a tic why have these all come to me when i am not very thankully not depressed,Mengapa saya memiliki begitu banyak gejala stres tidak mampu mengelola kemarahan saya motivasi saya naik dan turun tidak ingin melakukan apa-apa melakukan hal-hal acak dengan bagian-bagian tubuh saya sebagai tic mengapa semua ini datang kepada saya ketika saya tidak sangat berterima kasih tidak tertekan,why do i have so many symptoms of stress not being able to manage my anger my motivation goes up and down don't want to do anything random with my body parts as tic why all this comes to me when i'm not very grateful not to be depressed,Stress,3.294073e-01,0.947143
3,i know no break up is fun but you know what i mean without going into too much detail she was unhappy cheated on me broke up with me started dating other guy we also live together its been three weeks and the stress had caused my stomach to twist into knots i cannot untie not really sure where else got with this i just want to stop feeling like this,Aku tahu tidak ada putus yang menyenangkan tapi kau tahu apa yang saya maksud tanpa pergi ke terlalu banyak detail dia tidak bahagia ditipu pada saya mulai berkencan dengan pria lain kami juga hidup bersama sudah tiga minggu dan stres telah menyebabkan perut saya untuk memutar ke knot saya tidak bisa benar-benar yakin di mana lagi punya dengan ini saya hanya ingin berhenti merasa seperti ini,i know there's no breaking up that's fun but you know what i mean without going into too much detail she's unhappy cheated on me start dating another man we also live together already three weeks and stress has caused my stomach to twist into knots i can't really be sure where else i have with this i just want to stop feeling like this,Stress,4.010758e-01,0.936788
4,which forced me to acknowledge that i havent been performing to my own expectations enter the anxiety attack sobbing sh

In [41]:
# Save the augmented DataFrame
df_augmented.to_csv('/kaggle/working/augmented_Stress_fixx.csv', index=False)

In [42]:
# Menghapus semua tanda baca atau karakter non-alfabet dari kolom statement
df_augmented['statement'] = df_augmented['statement'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Menampilkan hasil setelah penghapusan tanda baca
df_augmented

,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,hey first time posting here i have been living in my car with my brother and cat since may i am feeling extremely depresseddefeated right now hence why im posting here ill start from the beginning,Hei pertama kali posting di sini saya telah tinggal di mobil saya dengan kakak saya dan kucing karena mungkin saya merasa sangat tertekan dikalahkan sekarang karena itu mengapa saya posting di sini sakit mulai dari awal,hey first time posting here ive been living in my car with my brother and cat because maybe im feeling very depressed being defeated right now because thats why im posting here sick start at the beginning,Stress,3.632574e-01,0.933453
1,i only realized this was a stupid idea in my freshman year of college when i started having flashbacks i tried once more to do therapy and almost failed out of school again i buried it all to be dealt with at a more convenient time i have now been working in a job that dont mind for the last year and months i found that november and december were a struggle and because of that i planned to address my stuff by going to therapy starting in,Aku hanya menyadari ini adalah ide bodoh di tahun pertama kuliahku ketika aku mulai mengalami kilas balik aku mencoba sekali lagi untuk melakukan terapi dan hampir gagal keluar dari sekolah lagi aku menguburnya semua untuk ditangani pada waktu yang lebih nyaman aku sekarang telah bekerja dalam pekerjaan yang tidak keberatan untuk tahun terakhir dan bulan aku menemukan bahwa noverber dan dekember adalah perjuangan dan karena itu saya berencana untuk mengatasi barang-barang saya dengan akan mulai terapi,i just realized this was a stupid idea in my first year of college when i started having flashbacks i tried once again to do therapy and almost failed out of school again i buried it all for a more comfortable time,Stress,1.601767e-01,0.919187
2,why do i have so many symptoms for stress not being able to manage my anger my motivation going up and down not wanting to do anything doing random things with parts of my body as a tic why have these all come to me when i am not very thankully not depressed,Mengapa saya memiliki begitu banyak gejala stres tidak mampu mengelola kemarahan saya motivasi saya naik dan turun tidak ingin melakukan apa-apa melakukan hal-hal acak dengan bagian-bagian tubuh saya sebagai tic mengapa semua ini datang kepada saya ketika saya tidak sangat berterima kasih tidak tertekan,why do i have so many symptoms of stress not being able to manage my anger my motivation goes up and down dont want to do anything random with my body parts as tic why all this comes to me when im not very grateful not to be depressed,Stress,3.294073e-01,0.947143
3,i know no break up is fun but you know what i mean without going into too much detail she was unhappy cheated on me broke up with me started dating other guy we also live together its been three weeks and the stress had caused my stomach to twist into knots i cannot untie not really sure where else got with this i just want to stop feeling like this,Aku tahu tidak ada putus yang menyenangkan tapi kau tahu apa yang saya maksud tanpa pergi ke terlalu banyak detail dia tidak bahagia ditipu pada saya mulai berkencan dengan pria lain kami juga hidup bersama sudah tiga minggu dan stres telah menyebabkan perut saya untuk memutar ke knot saya tidak bisa benar-benar yakin di mana lagi punya dengan ini saya hanya ingin berhenti merasa seperti ini,i know theres no breaking up thats fun but you know what i mean without going into too much detail shes unhappy cheated on me start dating another man we also live together already three weeks and stress has caused my stomach to twist into knots i cant really be sure where else i have with this i just want to stop feeling like this,Stress,4.010758e-01,0.936788
4,which forced me to acknowledge that i havent been performing to my own expectations enter the anxiety attack sobbing shaking fidgeting 

In [43]:
# Menyisakan hanya kolom 'statement' dan 'status'
df_bipolarrr = df_augmented.loc[:, ['statement', 'status']]

# Menampilkan DataFrame hasil
df_bipolarrr

,statement,status
0,hey first time posting here ive been living in my car with my brother and cat because maybe im feeling very depressed being defeated right now because thats why im posting here sick start at the beginning,Stress
1,i just realized this was a stupid idea in my first year of college when i started having flashbacks i tried once again to do therapy and almost failed out of school again i buried it all for a more comfortable time,Stress
2,why do i have so many symptoms of stress not being able to manage my anger my motivation goes up and down dont want to do anything random with my body parts as tic why all this comes to me when im not very grateful not to be depressed,Stress
3,i know theres no breaking up thats fun but you know what i mean without going into too much detail shes unhappy cheated on me start dating another man we also live together already three weeks and stress has caused my stomach to twist into knots i cant really be sure where else i have with this i just want to stop feeling like this,Stress
4,that forces me to admit that i havent done for my own hope entered the anxiety attacks crying is trembling and rambling up its all off i think that i dont need a safety net i think im done with all this reality proving otherwise,Stress
...,...,...
1396,he had been opposed to smoking weed as long as he was there when it happened in me to smoke with only him or when both of them seemed to have moved to the attitude where he was doing well with me smoking weed as long as he was there when it happened as in i could smoke with just him or when both of them were hanging out with the same friends ive tried to ask him and really understood why he felt like this but he couldnt rationally describe that it was the gut that it was that he could be that he could get in that he could legitimately that he could get disturbed and gave him a legitimate concern about it and almost noticed that he wanted to know why it so much but it couldnt help him try to explain how it was to explain how i didnt know how i wanted to explain how i wanted to stop to smoke but i wanted to explain the thing because i wanted to explain it was about this,Stress
1397,that really threw me i dont know why i felt so hard to overcome i cried often for some reason and i dont know why i havent done the whole uncontrolled crying thing since my last episode which was all the way back in novermber,Stress
1398,ive just had this stress stress my stress i just got x is worse i am gonna check myself for myself,Stress
1399,even if thats true and i believe her i hate it ive tried to explain my point of view and why i hate them still talking but whenever ive been raised we always end up arguing she says i have no reason to be jealous of her and that shes very happy with me and that im excessive im i dont try to make trouble when no one but i feel like i never want to tell her to stop talking to a friend but their history makes me uncomfortable,Stress


In [44]:
# Save the augmented DataFrame
df_bipolarrr.to_csv('/kaggle/working/dataBT_Stress_fixx.csv', index=False)

## Personality Disorder ##

In [45]:
import torch
# Pilih GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model names for translation
first_model_name = 'Helsinki-NLP/opus-mt-en-id'
second_model_name = 'Helsinki-NLP/opus-mt-id-en'

# Load the tokenizers
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained models and move them to GPU
first_model = MarianMTModel.from_pretrained(first_model_name).to(device)
second_model = MarianMTModel.from_pretrained(second_model_name).to(device)

In [46]:
# Filter the data for 'Depression' status
data_bipolar = data[data['status'] == 'Personality disorder']
original_texts = data_bipolar['statement'].tolist()
data_bipolar

,statement,status
13,so tired of everything i am so goddamn tired of everything i am not looking forward to anything have no goalsjust a lowpaid job that i dont really like i guess i can be proud that i got one in the first place despite how horribly painful that was to get used to it and my thoughts about quitting it after first week but honestly other than that i have no clue why i am doing all this i mean everythinggoing to job eating trying to do somethingliving i am so tired i barely drag my body to work or practically anywhere about my jobi got more or less used to it but i still feel like my coworkers hate me its my avpd brain acting up again goddamn pesky little thing theyre nice to me but this irrational part of me wins every time every day is a misery and i am so tired i always tell myself that i have to go forward and dont stop but by now i am not sure why or where i am going feels like i am just aimlessly wandering around in the darkness with no clue whatsoever,Personality disorder
109,its when we realize that were broken that the hope for fixing ourselves sets in recognize the problem folkstake control of it as best as you can,Personality disorder
114,i really need to go to the pharmacy please someone to encourage me help me its going to get worse if i dont go its been more than a week alreadyi hate that im neglecting and selfisolating myself so badly because of my issues i hate it so much im so lonely the only motivation i have is from myself and needless to say theres literally noneincidentally i have to see a doctor a dentist an orthodontist and an endocrinologist and its so so so hard to get myself to do it oh god im like slowly decaying,Personality disorder
147,liking someone i wanna know how other people with avpd are when you like someonehave a crushpersonally i find it very painful i wonder if i feel too strongly the feeling is intense and i dont know how normal that is not talking to the person i like feels more painful than it should be maybe its an avpd thing ive heard it can be like this for people with bpd i have several bpd traits and im not too sure how it works but maybe thats why i feel like this,Personality disorder
149,those that smoke or vape does it helphurt cigarette smoke i find gross but some of these new vape things smell quite nice and fruity and im somewhat intrigued about it just wondering if anyone does it and whether its helped or hurt their ability to cope with this curse,Personality disorder
...,...,...
14874,what even is the point ive been considering just killing myself already my life is empty anyway i dont have anything i dont have anyone i never did at this point it feels like im just procrastinating my death,Personality disorder
14875,i cant do it anymore im alone with my struggles and it has reached a point where its only suffering unfortunately i dont really have someone i can tell this so this will be my venting space right now the only thing currently holding me alive is my family if they didnt care and love me i would have kms years ago but now i really wish theyd just abandon me and stop caring because i really dont want to live anymore really i hate to say this but their love just makes it worse what do i owe them but yeah the thought of how theyd react if i did it is so bad that it currently holds me back i do have friends but they either never reach out to me or if they do its in a context of meeting as a group and that happens like times a year some of those people do stuff together i came to know but not once was i asked to join and i have a best friend whos just weird idk why i even call him my best friend maybe because i know him since years and we did stuff together he as well never reaches out by himself and when i do almost always declines is busy on those rare occasions we meet i never get asked a question no one ever showed interest in me besides my ex girlfriend but thats another story i dont really have a perspective because i was lazy struggled in school a

In [47]:
from tqdm import tqdm

# Pilih teks secara acak
num_texts_to_add = 526
selected_texts = random.sample(original_texts, num_texts_to_add)

# Fungsi untuk format teks ke MarianMT
def format_batch_texts(language_code, batch_texts):
    return [">>{}<< {}".format(language_code, text) for text in batch_texts]

# Fungsi untuk menerjemahkan dengan MarianMT
def perform_translation(texts, model, tokenizer, language="id"):
    formatted_texts = format_batch_texts(language, texts)
    inputs = tokenizer(formatted_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Fungsi back translation dengan batas re-translation satu kali jika hasil pertama duplikat
def perform_back_translation(texts, original_language="en", temporary_language="id", batch_size=32):
    back_translated = []
    tmp_translations = []
    
    pbar = tqdm(total=len(texts), desc="Back translation progress", dynamic_ncols=True)
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Translate to temporary language
        tmp_translated = perform_translation(batch_texts, first_model, first_model_tkn, temporary_language)
        tmp_translations.extend(tmp_translated)
        
        # Translate back to original language
        batch_back_translated = perform_translation(tmp_translated, second_model, second_model_tkn, original_language)
        
        # Cek duplikasi dan lakukan re-translation hanya sekali jika perlu
        final_batch = []
        for j, back_text in enumerate(batch_back_translated):
            if back_text == batch_texts[j] or back_text in back_translated:
                # Lakukan re-translation hanya sekali
                tmp_retranslated = perform_translation([batch_texts[j]], first_model, first_model_tkn, temporary_language)
                back_text_retranslated = perform_translation(tmp_retranslated, second_model, second_model_tkn, original_language)[0]
                
                # Jika hasil berbeda dan belum ada di dataset, gunakan
                if back_text_retranslated != batch_texts[j] and back_text_retranslated not in back_translated:
                    final_batch.append(back_text_retranslated)
                else:
                    final_batch.append(back_text)  # Jika tetap sama, gunakan hasil awal
            else:
                final_batch.append(back_text)
        
        back_translated.extend(final_batch)
        
        # Update progress bar
        pbar.update(len(batch_texts))
        pbar.set_description(f"Back translation progress: {len(back_translated)}/{len(texts)} teks diproses")

    pbar.close()
    print(f"\nProses back translation selesai! Total teks yang diproses: {len(back_translated)}")
    return back_translated, tmp_translations

# Jalankan back translation dengan batch size 32
back_translated_texts, tmp_translated_texts = perform_back_translation(selected_texts, batch_size=32)

# Fungsi untuk membersihkan teks
def clean_text(text):
    return text.lower()

# Bersihkan teks yang telah diperoleh
cleaned_texts = [clean_text(text) for text in back_translated_texts]

# Buat DataFrame baru dengan hasil augmentasi
df_augmented = pd.DataFrame({
    'original_text': selected_texts,
    'temporary_translation': tmp_translated_texts,
    'statement': cleaned_texts,
    'status': 'Personality disorder'
})

# Tampilkan hasil
df_augmented

Back translation progress: 526/526 teks diproses: 100%|██████████| 526/526 [16:10<00:00,  1.85s/it]


Proses back translation selesai! Total teks yang diproses: 526


,original_text,temporary_translation,statement,status
0,social days do you guys ever have those days where you talk more to people laugh more and just be happier in general a day where you dont feel so dead so unacceptable i have those days at times and then it goes back to the way it was being scared of people my confidence dropping low again and feeling like a void of loneliness,Hari-hari sosial yang pernah kalian miliki hari-hari di mana Anda berbicara lebih banyak untuk orang tertawa lebih banyak dan hanya lebih bahagia pada umumnya hari di mana Anda tidak merasa begitu mati sehingga tidak dapat diterima saya memiliki hari-hari pada kadang-kadang dan kemudian kembali ke cara itu menjadi takut dari orang-orang kepercayaan diri saya menjatuhkan rendah lagi dan merasa seperti kekosongan kesepian,the social days that you guys have had the days where you talk more to people laugh more and just be happier in general the days in which you don't feel so dead that it's unacceptable i have days in sometimes and then go back to the way it became afraid of people my confidence dropped low again and felt like a lonely void,Personality disorder
1,trying to wrap my head around all of this hear me out if you will so im m diagnosed adult with high functioning autism i have been lurking the schizoid community for some time because i have so much in common with the guys who post over there mostly its the anhedonia that really strikes a chord with methe prevailing attitude over at rschizoid seems to be this cavalier stance that loneliness is just a preference well i take issue with this myself ive been isolating on and off so much since even before puberty sure it kind of feels good to not have to spend cognitive energy on social stuff but loneliness is taking a toll on me my brain operates on use it or lose it and i feel that now more than ever especially since covidthe more i isolate which is something i can do day to day without sacrificing too much in the short run the less high functioning i become i guess i substitute beer vidyas and parasocially engaging on youtube for the real thing its kind of working but i really miss intimacy and that human connection i feel like im ever so slowly spiraling into insanity,Saya telah mengintai komunitas schizoid untuk beberapa waktu karena saya memiliki begitu banyak kesamaan dengan orang-orang yang posting di atas sana sebagian besar anhedonia yang benar-benar menyerang akord dengan saya sikap yang berlaku atas di rschizoid tampaknya sikap ini caviier bahwa kesepian adalah baik saya telah mengambil masalah dengan diri saya sendiri dan off begitu banyak sejak yakin itu jenis baik untuk tidak harus menghabiskan pada hal-hal sosial tapi adalah mengambil otak saya akan kehilangan dan sekarang saya merasa lebih terisolasi daripada hari yang saya benar-benar dapat melakukan hal yang lebih sedikit,i've been stalking schizoid communities for some time because i have so much in common with the people who post up there most of the anhedonia that really attacks the chords with my attitude that applies to this schizoid attitude that loneliness seems to be good i've taken matters with myself and off so much since sure it's kind of good not to have to spend on social things but is taking my brain will lose and now i feel more isolated than the day that i can actually do a little bit more,Personality disorder
2,people get frustrated by me do people get frustrated by you i dont know why but i detect frustration when im around others sometimes i just want to move far away and start a new life i just want to get away from it all and disappear they dont accept me and so i just feel like im better off alone for eternity i want to live on a remote island people just judge me for the way that i am and i hate iti dont know why they hate me so much,Orang-orang menjadi frustrasi oleh saya apakah orang-orang mendapatkan frustrasi oleh Anda saya tidak tahu mengapa tetapi saya mendeteksi frustrasi ketika Im sekitar orang lain 

In [48]:
# Mengatur agar teks panjang ditampilkan utuh
pd.set_option('display.max_colwidth', None)

# Menampilkan satu baris teks dari DataFrame untuk ketiga kolom
print(df_augmented.iloc[0])  # Menampilkan seluruh baris pertama

original_text                                                                                                            social days do you guys ever have those days where you talk more to people laugh more and just be happier in general a day where you dont feel so dead so unacceptable i have those days at times and then it goes back to the way it was being scared of people my confidence dropping low again and feeling like a void of loneliness
temporary_translation    Hari-hari sosial yang pernah kalian miliki hari-hari di mana Anda berbicara lebih banyak untuk orang tertawa lebih banyak dan hanya lebih bahagia pada umumnya hari di mana Anda tidak merasa begitu mati sehingga tidak dapat diterima saya memiliki hari-hari pada kadang-kadang dan kemudian kembali ke cara itu menjadi takut dari orang-orang kepercayaan diri saya menjatuhkan rendah lagi dan merasa seperti kekosongan kesepian
statement                                                                                             

In [49]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score

# ==================== BLEU Score Without Smoothing ====================
def calculate_bleu_score_no_smoothing(reference_texts, translated_texts):
    bleu_scores = []
    for ref, trans in zip(reference_texts, translated_texts):
        ref_tokens = ref.split()  # Tokenize reference
        trans_tokens = trans.split()  # Tokenize translation
        bleu_score = sentence_bleu([ref_tokens], trans_tokens)  # BLEU score without smoothing
        bleu_scores.append(bleu_score)
    return bleu_scores

bleu_scores_no_smoothing = calculate_bleu_score_no_smoothing(selected_texts, back_translated_texts)

# ==================== BERT Score ====================
def calculate_bert_score(reference_texts, translated_texts, lang="en"):
    P, R, F1 = score(translated_texts, reference_texts, lang=lang)
    return P, R, F1

P, R, F1 = calculate_bert_score(selected_texts, back_translated_texts)

# Add scores to the DataFrame
df_augmented['BLEU_Score'] = bleu_scores_no_smoothing
df_augmented['BERT_F1'] = F1.tolist()

# Calculate average scores
average_bleu = df_augmented['BLEU_Score'].mean()
average_bert_f1 = df_augmented['BERT_F1'].mean()

# Display average scores
print(f"\nAverage BLEU Score: {average_bleu:.4f}")
print(f"Average BERT F1 Score: {average_bert_f1:.4f}")

df_augmented

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Average BLEU Score: 0.2098
Average BERT F1 Score: 0.8773


,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,social days do you guys ever have those days where you talk more to people laugh more and just be happier in general a day where you dont feel so dead so unacceptable i have those days at times and then it goes back to the way it was being scared of people my confidence dropping low again and feeling like a void of loneliness,Hari-hari sosial yang pernah kalian miliki hari-hari di mana Anda berbicara lebih banyak untuk orang tertawa lebih banyak dan hanya lebih bahagia pada umumnya hari di mana Anda tidak merasa begitu mati sehingga tidak dapat diterima saya memiliki hari-hari pada kadang-kadang dan kemudian kembali ke cara itu menjadi takut dari orang-orang kepercayaan diri saya menjatuhkan rendah lagi dan merasa seperti kekosongan kesepian,the social days that you guys have had the days where you talk more to people laugh more and just be happier in general the days in which you don't feel so dead that it's unacceptable i have days in sometimes and then go back to the way it became afraid of people my confidence dropped low again and felt like a lonely void,Personality disorder,0.391874,0.932071
1,trying to wrap my head around all of this hear me out if you will so im m diagnosed adult with high functioning autism i have been lurking the schizoid community for some time because i have so much in common with the guys who post over there mostly its the anhedonia that really strikes a chord with methe prevailing attitude over at rschizoid seems to be this cavalier stance that loneliness is just a preference well i take issue with this myself ive been isolating on and off so much since even before puberty sure it kind of feels good to not have to spend cognitive energy on social stuff but loneliness is taking a toll on me my brain operates on use it or lose it and i feel that now more than ever especially since covidthe more i isolate which is something i can do day to day without sacrificing too much in the short run the less high functioning i become i guess i substitute beer vidyas and parasocially engaging on youtube for the real thing its kind of working but i really miss intimacy and that human connection i feel like im ever so slowly spiraling into insanity,Saya telah mengintai komunitas schizoid untuk beberapa waktu karena saya memiliki begitu banyak kesamaan dengan orang-orang yang posting di atas sana sebagian besar anhedonia yang benar-benar menyerang akord dengan saya sikap yang berlaku atas di rschizoid tampaknya sikap ini caviier bahwa kesepian adalah baik saya telah mengambil masalah dengan diri saya sendiri dan off begitu banyak sejak yakin itu jenis baik untuk tidak harus menghabiskan pada hal-hal sosial tapi adalah mengambil otak saya akan kehilangan dan sekarang saya merasa lebih terisolasi daripada hari yang saya benar-benar dapat melakukan hal yang lebih sedikit,i've been stalking schizoid communities for some time because i have so much in common with the people who post up there most of the anhedonia that really attacks the chords with my attitude that applies to this schizoid attitude that loneliness seems to be good i've taken matters with myself and off so much since sure it's kind of good not to have to spend on social things but is taking my brain will lose and now i feel more isolated than the day that i can actually do a little bit more,Personality disorder,0.072047,0.847841
2,people get frustrated by me do people get frustrated by you i dont know why but i detect frustration when im around others sometimes i just want to move far away and start a new life i just want to get away from it all and disappear they dont accept me and so i just feel like im better off alone for eternity i want to live on a remote island people just judge me for the way that i am and i hate iti dont know why they hate me so much,Orang-orang menjadi frustrasi oleh saya apakah orang-orang mendapatkan frustrasi oleh Anda saya tidak tahu mengapa tetapi 

In [50]:
# Save the augmented DataFrame
df_augmented.to_csv('/kaggle/working/augmented_PD_fixx.csv', index=False)

In [51]:
# Menghapus semua tanda baca atau karakter non-alfabet dari kolom statement
df_augmented['statement'] = df_augmented['statement'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Menampilkan hasil setelah penghapusan tanda baca
df_augmented

,original_text,temporary_translation,statement,status,BLEU_Score,BERT_F1
0,social days do you guys ever have those days where you talk more to people laugh more and just be happier in general a day where you dont feel so dead so unacceptable i have those days at times and then it goes back to the way it was being scared of people my confidence dropping low again and feeling like a void of loneliness,Hari-hari sosial yang pernah kalian miliki hari-hari di mana Anda berbicara lebih banyak untuk orang tertawa lebih banyak dan hanya lebih bahagia pada umumnya hari di mana Anda tidak merasa begitu mati sehingga tidak dapat diterima saya memiliki hari-hari pada kadang-kadang dan kemudian kembali ke cara itu menjadi takut dari orang-orang kepercayaan diri saya menjatuhkan rendah lagi dan merasa seperti kekosongan kesepian,the social days that you guys have had the days where you talk more to people laugh more and just be happier in general the days in which you dont feel so dead that its unacceptable i have days in sometimes and then go back to the way it became afraid of people my confidence dropped low again and felt like a lonely void,Personality disorder,0.391874,0.932071
1,trying to wrap my head around all of this hear me out if you will so im m diagnosed adult with high functioning autism i have been lurking the schizoid community for some time because i have so much in common with the guys who post over there mostly its the anhedonia that really strikes a chord with methe prevailing attitude over at rschizoid seems to be this cavalier stance that loneliness is just a preference well i take issue with this myself ive been isolating on and off so much since even before puberty sure it kind of feels good to not have to spend cognitive energy on social stuff but loneliness is taking a toll on me my brain operates on use it or lose it and i feel that now more than ever especially since covidthe more i isolate which is something i can do day to day without sacrificing too much in the short run the less high functioning i become i guess i substitute beer vidyas and parasocially engaging on youtube for the real thing its kind of working but i really miss intimacy and that human connection i feel like im ever so slowly spiraling into insanity,Saya telah mengintai komunitas schizoid untuk beberapa waktu karena saya memiliki begitu banyak kesamaan dengan orang-orang yang posting di atas sana sebagian besar anhedonia yang benar-benar menyerang akord dengan saya sikap yang berlaku atas di rschizoid tampaknya sikap ini caviier bahwa kesepian adalah baik saya telah mengambil masalah dengan diri saya sendiri dan off begitu banyak sejak yakin itu jenis baik untuk tidak harus menghabiskan pada hal-hal sosial tapi adalah mengambil otak saya akan kehilangan dan sekarang saya merasa lebih terisolasi daripada hari yang saya benar-benar dapat melakukan hal yang lebih sedikit,ive been stalking schizoid communities for some time because i have so much in common with the people who post up there most of the anhedonia that really attacks the chords with my attitude that applies to this schizoid attitude that loneliness seems to be good ive taken matters with myself and off so much since sure its kind of good not to have to spend on social things but is taking my brain will lose and now i feel more isolated than the day that i can actually do a little bit more,Personality disorder,0.072047,0.847841
2,people get frustrated by me do people get frustrated by you i dont know why but i detect frustration when im around others sometimes i just want to move far away and start a new life i just want to get away from it all and disappear they dont accept me and so i just feel like im better off alone for eternity i want to live on a remote island people just judge me for the way that i am and i hate iti dont know why they hate me so much,Orang-orang menjadi frustrasi oleh saya apakah orang-orang mendapatkan frustrasi oleh Anda saya tidak tahu mengapa tetapi saya 

In [52]:
# Menyisakan hanya kolom 'statement' dan 'status'
df_bipolarrr = df_augmented.loc[:, ['statement', 'status']]

# Menampilkan DataFrame hasil
df_bipolarrr

,statement,status
0,the social days that you guys have had the days where you talk more to people laugh more and just be happier in general the days in which you dont feel so dead that its unacceptable i have days in sometimes and then go back to the way it became afraid of people my confidence dropped low again and felt like a lonely void,Personality disorder
1,ive been stalking schizoid communities for some time because i have so much in common with the people who post up there most of the anhedonia that really attacks the chords with my attitude that applies to this schizoid attitude that loneliness seems to be good ive taken matters with myself and off so much since sure its kind of good not to have to spend on social things but is taking my brain will lose and now i feel more isolated than the day that i can actually do a little bit more,Personality disorder
2,the people get frustrated by me whether people get frustrated by you i dont know why but i detect frustration when im around other people sometimes i just want to move away and start a new life i just want to get away from it all and disappear they dont accept me and so i just feel better forever,Personality disorder
3,the kings army of poets what do you get if youve taken a test i bet a lot of ppl here will get gtkinglt i wantedexpected gtpoetlt but i got that description feel like a personal attack loloheres link if anyone wants to,Personality disorder
4,is it better to never experience love and thus never get hurt or be worth a try even if the wound will have lasting consequences or will cause something more drastic like suicide as the title says im so sensitive to the rejection that the smallest amount of banter dessert or subtle ridicule can send me to a suicide episode by never going to become quite the spell in its core im presented with a chance of entering the relationship but im really afraid shitlessugh just wants to pull away,Personality disorder
...,...,...
521,is the anxiety playing part in your avpd i have anxiety disorder that one becomes social anxiety so ive had a lot of anxiety with my social situation when it comes to the social situation,Personality disorder
522,my problem with avpd is that it makes me weak the root of an avpd and my life of my mind has been my problem with avpd is that it makes me weak i feel like a prey living among predators,Personality disorder
523,what other people struggle with internet addiction have you found nothing to help with that because of avoiding and social anxiety ive used more and more internet to roll up instagrams and reddits and just wasting his time like being social without the stress of socialization but its not good for my longterm mental health and worsening i find anyones trick or tips to get out of it causing socialization is not the best solution but which is not easy to avoid and anxiety,Personality disorder
524,is it because my partner is selfish or because appd is going to take and take over and ask for more and my country i would never do a good gesture for me like offering to take me lunch take me to be any affection that really treats me like a doormat cancels the last minute of commitment with random people ignoring my feelings and our place will be used as a place as a place of business with a guest room all day long and never take any criticism and see how much behavior is rooted in itself but see how some people above the same things that he sees above as he sees some people who have similar views of themselves,Personality disorder


In [53]:
# Save the augmented DataFrame
df_bipolarrr.to_csv('/kaggle/working/dataBT_PD_fixx.csv', index=False)